## PRIMER PASO: BARAJEAR DATOS

## SETUP - Instalación de Dependencias

In [ ]:
# Instalación de librerías necesarias
!pip install deep-translator pandas tqdm seaborn nltk scikit-learn gensim matplotlib numpy transformers torch

print("✓ Todas las dependencias instaladas correctamente")

In [ ]:
# Crear carpetas para organizar los archivos del proyecto
import os

folders = ['data', 'data_processed', 'models', 'charts']

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    
print("✓ Estructura de carpetas creada:")
print("  📁 data/              - Datasets originales")
print("  📁 data_processed/    - Datos procesados (CSV)")
print("  📁 models/            - Modelos entrenados")
print("  📁 charts/            - Gráficos generados")

## Mezclado

In [ ]:
import pandas as pd
import random

# Cargar dataset original
df = pd.read_csv('data/initial_data.csv')

print(f"Dataset original cargado: {len(df)} noticias")
print(f"Distribución de sentimientos:")
print(df['Sentiment'].value_counts())
print("\n" + "="*60)

# Crear copia para noticias correctas (50%)
df_correctas = df.copy()
df_correctas['Etiqueta'] = 'correcta'

# Separar noticias por sentimiento
df_positive = df[df['Sentiment'] == 'positive']
df_negative = df[df['Sentiment'] == 'negative']

print(f"\nNoticias positivas disponibles: {len(df_positive)}")
print(f"Noticias negativas disponibles: {len(df_negative)}")

# Extraer fragmentos de una noticia dividiendo por comas o puntos
def extraer_fragmentos(noticia):
    fragmentos = [p.strip() for p in noticia.split(',') if p.strip()]
    
    if len(fragmentos) <= 1:
        fragmentos = [p.strip() for p in noticia.split('.') if p.strip()]
    
    return fragmentos

# Crear noticias incorrectas mezclando fragmentos de sentimientos opuestos
def crear_mezclas_opuestas(df_positive, df_negative, num_mezclas):
    noticias_mezcladas = []
    
    print(f"\nCreando {num_mezclas} mezclas de sentimientos opuestos...")
    
    for i in range(num_mezclas):
        if (i + 1) % 1000 == 0:
            print(f"  Procesadas {i + 1}/{num_mezclas}...")
        
        # Seleccionar una noticia positiva y una negativa aleatoriamente
        idx_pos = random.randint(0, len(df_positive) - 1)
        idx_neg = random.randint(0, len(df_negative) - 1)
        
        noticia_pos = df_positive.iloc[idx_pos]['Sentence']
        noticia_neg = df_negative.iloc[idx_neg]['Sentence']
        
        # Extraer fragmentos de cada noticia
        frag_pos = extraer_fragmentos(noticia_pos)
        frag_neg = extraer_fragmentos(noticia_neg)
        
        # Seleccionar 1-2 fragmentos de cada noticia
        if len(frag_pos) > 0 and len(frag_neg) > 0:
            num_pos = min(random.randint(1, 2), len(frag_pos))
            num_neg = min(random.randint(1, 2), len(frag_neg))
            
            fragmentos_seleccionados = (
                random.sample(frag_pos, num_pos) + 
                random.sample(frag_neg, num_neg)
            )
            
            # Mezclar el orden de los fragmentos
            random.shuffle(fragmentos_seleccionados)
            
            noticia_mezclada = ', '.join(fragmentos_seleccionados)
            if noticia_mezclada and noticia_mezclada[-1] not in '.!?':
                noticia_mezclada += '.'
            
            # Asignar sentimiento aleatorio
            sentiment = random.choice(['positive', 'negative'])
            
            noticias_mezcladas.append({
                'Sentence': noticia_mezclada,
                'Sentiment': sentiment,
                'Etiqueta': 'incorrecta'
            })
    
    return pd.DataFrame(noticias_mezcladas)

# Crear el mismo número de noticias incorrectas que correctas
num_mezclas = len(df_correctas)
df_incorrectas = crear_mezclas_opuestas(df_positive, df_negative, num_mezclas)

# Combinar datasets
df_final = pd.concat([df_correctas, df_incorrectas], ignore_index=True)

# Mezclar aleatoriamente
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar estadísticas
print(f"\n{'='*60}")
print(f"ESTADÍSTICAS DEL DATASET FINAL")
print(f"{'='*60}")
print(f"Total de filas: {len(df_final)}")
print(f"Noticias correctas: {len(df_final[df_final['Etiqueta'] == 'correcta'])} ({len(df_final[df_final['Etiqueta'] == 'correcta'])/len(df_final)*100:.1f}%)")
print(f"Noticias incorrectas: {len(df_final[df_final['Etiqueta'] == 'incorrecta'])} ({len(df_final[df_final['Etiqueta'] == 'incorrecta'])/len(df_final)*100:.1f}%)")

print(f"\nDistribución de sentimientos en noticias CORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'correcta']['Sentiment'].value_counts())

print(f"\nDistribución de sentimientos en noticias INCORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'incorrecta']['Sentiment'].value_counts())

# Mostrar ejemplos
print(f"\n{'='*60}")
print(f"EJEMPLOS")
print(f"{'='*60}")

print("\n[NOTICIA CORRECTA]")
ejemplo_correcta = df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentiment']}")
print(ejemplo_correcta[:300] + ("..." if len(ejemplo_correcta) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 1]")
ejemplo_inc1 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentiment']}")
print(ejemplo_inc1[:300] + ("..." if len(ejemplo_inc1) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 2]")
ejemplo_inc2 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentiment']}")
print(ejemplo_inc2[:300] + ("..." if len(ejemplo_inc2) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 3]")
ejemplo_inc3 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentiment']}")
print(ejemplo_inc3[:300] + ("..." if len(ejemplo_inc3) > 300 else ""))

# Guardar dataset final
output_path = 'data/dataset_mezclado_final.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8')
print(f"\n{'='*60}")
print(f"✓ Dataset guardado como '{output_path}'")
print(f"{'='*60}")

## TRADUCCIÓN

In [ ]:
# Traducir el dataset a múltiples idiomas usando Google Translator
import pandas as pd
from deep_translator import GoogleTranslator
import random
from tqdm import tqdm
import time
import os

# Cargar dataset
df = pd.read_csv('data/dataset_mezclado_final.csv')

# Idiomas disponibles
idiomas = ['en', 'fr', 'de', 'it', 'pt', 'ca', 'eu', 'gl']

nombres_idiomas = {
    'es': 'español',
    'en': 'inglés',
    'fr': 'francés',
    'de': 'alemán',
    'it': 'italiano',
    'pt': 'portugués',
    'ca': 'catalán',
    'eu': 'euskera',
    'gl': 'gallego'
}

df_traducido = df.copy()
df_traducido['Idioma'] = ''

# Traducir texto con manejo de errores
def traducir_texto(texto, idioma_destino):
    try:
        if idioma_destino == 'es':
            return texto
        translator = GoogleTranslator(source='es', target=idioma_destino)
        traduccion = translator.translate(texto)
        time.sleep(0.5)
        return traduccion
    except Exception as e:
        print(f"Error traduciendo a {idioma_destino}: {e}")
        return texto

# Traducir solo la columna 'Sentence' a un idioma aleatorio por fila
print("Iniciando traducción del dataset...")
print(f"Total de filas a procesar: {len(df_traducido)}")

for idx in tqdm(range(len(df_traducido))):
    # Seleccionar idioma aleatorio (mayor probabilidad para español)
    idioma_elegido = random.choice(idiomas + ['es', 'es', 'es'])
    
    # Traducir la columna 'Sentence'
    texto_original = df_traducido.loc[idx, 'Sentence']
    df_traducido.loc[idx, 'Sentence'] = traducir_texto(texto_original, idioma_elegido)
    df_traducido.loc[idx, 'Idioma'] = nombres_idiomas[idioma_elegido]

# Guardar dataset traducido
output_path = 'data/dataset_multiidioma.csv'
df_traducido.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n✓ Dataset traducido guardado en: {output_path}")
print(f"Total de filas: {len(df_traducido)}")
print(f"\nEstructura del dataset:")
print(df_traducido.head())
print(f"\nDistribución de idiomas:")
print(df_traducido['Idioma'].value_counts())

## 1. Análisis Exploratorio de Datos (EDA)

In [ ]:
import pandas as pd
import numpy as np

print("Iniciando Análisis Exploratorio de Datos...")
try:
    df = pd.read_csv('data/dataset_multiidioma.csv', encoding='utf-8')
except FileNotFoundError:
    print("\nERROR: No se encontró 'dataset_multiidioma.csv'.")
    exit()

print("=" * 60)
print(" RESUMEN DEL DATASET ")
print("=" * 60)

# Mostrar información básica del dataset
print("\n1. ESTRUCTURA Y CONTEO DE VALORES NO NULOS:")
df.info()

# Estadísticas descriptivas
print("\n2. ESTADÍSTICAS DESCRIPTIVAS:")
print("-" * 60)
print(df.describe(include='all'))

# Distribuciones de variables categóricas
print("\n3. DISTRIBUCIONES CLAVE (Conteo y Porcentaje):")
for col in ['Sentiment', 'Etiqueta', 'Idioma']:
    counts = df[col].value_counts().rename('Conteo')
    percents = df[col].value_counts(normalize=True).mul(100).round(2).rename('Porcentaje (%)')
    print(f"\n--- Distribución de: {col} ---")
    print(pd.concat([counts, percents], axis=1))

# Calcular longitud de oraciones en palabras
df['sentence_length'] = df['Sentence'].str.split().str.len()
print("\n4. ESTADÍSTICAS DE LONGITUD DE ORACIONES (En palabras):")
print("-" * 60)
print(df['sentence_length'].describe().round(2))

# Análisis de valores nulos
print("\n5. ANÁLISIS DE VALORES NULOS:")
print("-" * 60)
nulos = df.isnull().sum()
if nulos.sum() == 0:
    print("✓ No hay valores nulos en el dataset")
else:
    print(nulos[nulos > 0])

# Análisis de duplicados
print("\n6. ANÁLISIS DE DUPLICADOS:")
print("-" * 60)
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados} ({duplicados/len(df)*100:.2f}%)")
if duplicados > 0:
    print(f"Filas únicas: {len(df) - duplicados}")

# Análisis cruzado entre variables
print("\n7. ANÁLISIS CRUZADO: Sentiment vs Etiqueta")
print("-" * 60)
crosstab = pd.crosstab(df['Sentiment'], df['Etiqueta'], margins=True)
print("\nConteo absoluto:")
print(crosstab)
print("\nPorcentaje por fila:")
crosstab_pct = pd.crosstab(df['Sentiment'], df['Etiqueta'], normalize='index') * 100
print(crosstab_pct.round(2))

## 2. Visualizaciones del Dataset

In [ ]:
# Crear visualizaciones del dataset
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set_theme(style="whitegrid", palette="husl")

print("=" * 60)
print("VISUALIZACIONES DEL DATASET")
print("=" * 60)

# 1. Distribuciones principales
print("\n1. DISTRIBUCIONES PRINCIPALES")
print("-" * 60)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Distribuciones del Dataset', fontsize=16, fontweight='bold')

# Gráfico de sentimientos
sns.countplot(x='Sentiment', data=df, ax=axes[0], order=df['Sentiment'].value_counts().index)
axes[0].set_title('Distribución de Sentimientos')
axes[0].set_xlabel('Sentimiento')
axes[0].set_ylabel('Frecuencia')

# Gráfico de idiomas (top 5)
top_idiomas = df['Idioma'].value_counts().head(5).index
df_top_idiomas = df[df['Idioma'].isin(top_idiomas)]
sns.countplot(data=df_top_idiomas, y='Idioma', order=top_idiomas, ax=axes[1], palette='viridis')
axes[1].set_title('Top 5 Idiomas')
axes[1].set_xlabel('Frecuencia')

# Gráfico de etiquetas
colors = ['#2ecc71', '#e74c3c']
sns.countplot(data=df, x='Etiqueta', ax=axes[2], palette=colors)
axes[2].set_title('Distribución de Etiquetas')
axes[2].set_xlabel('Etiqueta')
axes[2].set_ylabel('Frecuencia')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('charts/01_distribuciones.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 01_distribuciones.png")
plt.show()

# 2. Análisis cruzado
print("\n2. ANÁLISIS CRUZADO")
print("-" * 60)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
fig.suptitle('Relación Sentimiento vs Etiqueta', fontsize=16, fontweight='bold')

# Crear heatmap de porcentajes
pivot_pct = pd.crosstab(df['Sentiment'], df['Etiqueta'], normalize='index') * 100
sns.heatmap(pivot_pct, annot=True, fmt='.1f', cmap='RdYlGn_r', ax=ax, 
            cbar_kws={'label': 'Porcentaje (%)'})
ax.set_title('Porcentaje por Sentimiento')
ax.set_xlabel('Etiqueta')
ax.set_ylabel('Sentimiento')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('charts/02_heatmap_sentiment_etiqueta.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 02_heatmap_sentiment_etiqueta.png")
plt.show()

# 3. Análisis de longitud de oraciones
print("\n3. ANÁLISIS DE LONGITUD")
print("-" * 60)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Análisis de Longitud de Oraciones', fontsize=16, fontweight='bold')

# Histograma de longitudes
sns.histplot(df['sentence_length'], bins=30, kde=True, ax=axes[0])
media = df['sentence_length'].mean()
axes[0].axvline(media, color='red', linestyle='--', label=f'Media: {media:.1f}')
axes[0].set_title('Distribución de Longitud')
axes[0].set_xlabel('Número de Palabras')
axes[0].legend()

# Boxplot por sentimiento
sns.boxplot(x='Sentiment', y='sentence_length', data=df, ax=axes[1])
axes[1].set_title('Longitud por Sentimiento')
axes[1].set_xlabel('Sentimiento')
axes[1].set_ylabel('Número de Palabras')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.savefig('charts/03_analisis_longitud.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 03_analisis_longitud.png")
plt.show()

# Resumen
print("\n" + "=" * 60)
print("RESUMEN DE VISUALIZACIONES")
print("=" * 60)
print("✓ 01_distribuciones.png")
print("✓ 02_heatmap_sentiment_etiqueta.png")
print("✓ 03_analisis_longitud.png")

## 3. Preprocesamiento de Datos - Tokenización y Limpieza

In [ ]:
import re
import pandas as pd
import string
import nltk

# Descargar recursos de NLTK
try:
    print("Verificando recursos NLTK esenciales...")
    nltk.download('punkt', quiet=True, raise_on_error=False)
    nltk.download('stopwords', quiet=True, raise_on_error=False)
    print("✓ Recursos NLTK (punkt, stopwords) listos.")
except Exception:
    print("ATENCIÓN: La descarga de recursos de NLTK falló. Usaremos tokenización simple.")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

# Tokenización simple como alternativa
def simple_tokenize(text):
    return re.findall(r"[\w']+|[.,!?;]", text.lower())

print("\n" + "=" * 80)
print("PREPROCESAMIENTO DE DATOS")
print("=" * 80)

data = 'data/dataset_multiidioma.csv'

# Cargar stopwords en múltiples idiomas
try:
    STOPWORDS_ES = set(stopwords.words('spanish'))
    STOPWORDS_EN = set(stopwords.words('english'))
    STOPWORDS_ALL = STOPWORDS_ES.union(STOPWORDS_EN) 
except LookupError:
    STOPWORDS_ALL = set()

df_processed = pd.read_csv(data).copy()

# Preprocesar texto: limpieza, tokenización y eliminación de stopwords
def preprocess_text(text, stopwords_set, min_len=2):
    if pd.isna(text) or not str(text).strip():
        return []
    
    text = str(text)
    
    # Eliminar URLs y menciones
    text = re.sub(r'http\S+|www\S+|https\S+|@\w+|#\w+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenizar y convertir a minúsculas
    try:
        tokens = word_tokenize(text.lower()) 
    except LookupError:
        tokens = simple_tokenize(text)

    # Eliminar puntuación, stopwords y tokens cortos
    tokens_final = [
        token for token in tokens
        if token not in string.punctuation 
        and token not in stopwords_set
        and len(token) > min_len
    ]
    
    return tokens_final

# Aplicar preprocesamiento
print("\n1. APLICANDO PROCESO DE LIMPIEZA...")
print("-" * 80)

df_processed['tokens_processed'] = df_processed['Sentence'].apply(
    lambda x: preprocess_text(x, STOPWORDS_ALL)
)

df_processed['tokens_original'] = df_processed['Sentence'].apply(
     lambda x: simple_tokenize(str(x)) if not pd.isna(x) else []
)

print("✓ Preprocesamiento completado en la columna 'tokens_processed'")

# Mostrar resultados
print("\n" + "=" * 80)
print("RESULTADOS DEL PREPROCESAMIENTO")
print("=" * 80)

print("\n✅ Columna de tokens procesados (Primeras 5 filas):")
print(df_processed[['Sentence', 'tokens_processed']].head())

print("-" * 80)

# Ejemplo de transformación
if len(df_processed) > 0:
    print("\nEjemplo de transformación de la primera fila:")
    print(f"  Texto Original: {df['Sentence'].iloc[0]}")
    print(f"  Tokens Finales: {df_processed['tokens_processed'].iloc[0]}")

print("-" * 80)

# Calcular estadísticas de reducción
df_processed['num_tokens_original'] = df_processed['tokens_original'].apply(len)
df_processed['num_tokens_processed'] = df_processed['tokens_processed'].apply(len)

mean_original = df_processed['num_tokens_original'].mean()
mean_processed = df_processed['num_tokens_processed'].mean()
reduction_percentage = ((mean_original - mean_processed) / mean_original * 100) if mean_original > 0 else 0

print(f"\nEstadísticas de Longitud:")
print(f"  Promedio de tokens originales: {mean_original:.2f}")
print(f"  Promedio de tokens procesados: {mean_processed:.2f}")
print(f"  Reducción de ruido promedio: **{reduction_percentage:.2f}%**")

# Guardar resultado
df_processed.to_csv('data_processed/datos_preprocesados_simple.csv', index=False)
print(f"\n✓ Datos guardados en 'data_processed/datos_preprocesados_simple.csv'")

## 4. Lemmatization y Stemming

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
import nltk
import numpy as np
import ast

# Descargar recursos de NLTK
try:
    print("Verificando recursos NLTK esenciales (wordnet)...")
    nltk.download('wordnet', quiet=True, raise_on_error=False)
    print("✓ Recurso 'wordnet' listo.")
except Exception:
    print("ATENCIÓN: El recurso 'wordnet' de NLTK no está disponible.")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

print("\n" + "=" * 60)
print("LEMMATIZATION Y STEMMING")
print("=" * 60)

# Cargar datos
df_processed = pd.read_csv('data_processed/datos_preprocesados_simple.csv').copy()

# Convertir strings de listas a listas reales
print("\n1. CONVIRTIENDO TOKENS (STRING → LISTA)")
print("-" * 60)

def convert_to_list(value):
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            result = ast.literal_eval(value)
            if isinstance(result, list):
                return result
        except:
            return value.split()
    return []

# Aplicar conversión a las columnas de tokens
df_processed['tokens_processed'] = df_processed['tokens_processed'].apply(convert_to_list)
df_processed['tokens_original'] = df_processed['tokens_original'].apply(convert_to_list)

# Verificar conversión
sample = df_processed['tokens_processed'].iloc[0]
if isinstance(sample, list) and len(sample) > 0 and isinstance(sample[0], str):
    vocab_real = len(set(token for tokens in df_processed['tokens_processed'] for token in tokens))
    print(f"✓ Conversión exitosa")
    print(f"  Tipo: {type(sample)}")
    print(f"  Ejemplo: {sample[:5]}")
    print(f"  Vocabulario real: {vocab_real} palabras")
else:
    print("⚠️  ADVERTENCIA: La conversión puede tener problemas")

# Inicializar herramientas de stemming y lemmatization
porter_stemmer = PorterStemmer()
word_lemmatizer = WordNetLemmatizer()

# Aplicar stemming
print("\n2. STEMMING (Porter Stemmer)")
print("-" * 60)

df_processed['tokens_stemmed'] = df_processed['tokens_processed'].apply(
    lambda tokens: [porter_stemmer.stem(token) for token in tokens] if isinstance(tokens, list) else []
)
print("✓ Stemming completado")

# Aplicar lemmatization
print("\n3. LEMMATIZATION (WordNet Lemmatizer)")
print("-" * 60)

df_processed['tokens_lemmatized'] = df_processed['tokens_processed'].apply(
    lambda tokens: [word_lemmatizer.lemmatize(token, pos='v') for token in tokens] if isinstance(tokens, list) else []
)
print("✓ Lemmatization completada")

# Comparar resultados
print("\n4. COMPARACIÓN DE RESULTADOS")
print("-" * 60)

if not df_processed.empty and len(df_processed['tokens_processed'].iloc[0]) > 0:
    original = df_processed['tokens_processed'].iloc[0][:3]
    stemmed = df_processed['tokens_stemmed'].iloc[0][:3]
    lemmatized = df_processed['tokens_lemmatized'].iloc[0][:3]
    
    print(f"{'Token Original':<20} {'Stemming':<20} {'Lemmatization':<20}")
    print("-" * 60)
    for i in range(min(len(original), 3)):
        print(f"{original[i]:<20} {stemmed[i]:<20} {lemmatized[i]:<20}")

# Calcular tamaño de vocabulario después de cada técnica
vocab_original = set(token for tokens in df_processed['tokens_processed'] for token in tokens if isinstance(tokens, list))
vocab_stemmed = set(token for tokens in df_processed['tokens_stemmed'] for token in tokens if isinstance(tokens, list))
vocab_lemmatized = set(token for tokens in df_processed['tokens_lemmatized'] for token in tokens if isinstance(tokens, list))

print(f"\nTamaño del vocabulario base: {len(vocab_original)}")
print(f"Tamaño después de Stemming: {len(vocab_stemmed)} ({((len(vocab_original) - len(vocab_stemmed)) / len(vocab_original) * 100):.2f}% reducción)")
print(f"Tamaño después de Lemmatization: {len(vocab_lemmatized)} ({((len(vocab_original) - len(vocab_lemmatized)) / len(vocab_original) * 100):.2f}% reducción)")

# Convertir tokens a texto para compatibilidad
df_processed['text_stemmed'] = df_processed['tokens_stemmed'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else ''
)
df_processed['text_lemmatized'] = df_processed['tokens_lemmatized'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else ''
)
df_processed['text_processed_base'] = df_processed['tokens_processed'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else ''
)

# Crear columna de tokens limpios SIN lematizar (para embeddings pre-entrenados)
df_processed['tokens_clean'] = df_processed['tokens_processed'].copy()

# Convertir a texto también
df_processed['text_clean'] = df_processed['tokens_clean'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else ''
)

print("\n✓ Columna 'tokens_clean' creada para embeddings (sin lematizar)")

# Guardar
df_processed.to_csv('data_processed/datos_preprocesados_completo.csv', index=False)
print(f"\n✓ Datos guardados en 'data_processed/datos_preprocesados_completo.csv'")

## 5. Representación Tradicional: Bag of Words (BoW)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="viridis")

print("\n" + "=" * 60)
print("VECTORIZACIÓN: BAG OF WORDS (BoW)")
print("=" * 60)

# Seleccionar columna de texto procesado
TEXT_COLUMN = 'text_lemmatized'

# Crear vectorizador con parámetros para reducir ruido
bow_vectorizer = CountVectorizer(
    max_features=5000, 
    min_df=2, 
    max_df=0.8, 
    ngram_range=(1, 2)
)

# Crear matriz BoW
X_bow = bow_vectorizer.fit_transform(df_processed[TEXT_COLUMN])
feature_names = bow_vectorizer.get_feature_names_out()

print(f"✓ Matriz BoW creada. Forma: {X_bow.shape}")
print(f"Tamaño del Vocabulario Final: {X_bow.shape[1]}")

# Calcular frecuencias de términos
print("\n2. ANÁLISIS DE FRECUENCIAS (TOP 15)")
print("-" * 60)

word_freq = np.asarray(X_bow.sum(axis=0)).flatten()
freq_df = pd.DataFrame({
    'word': feature_names,
    'frequency': word_freq
}).sort_values('frequency', ascending=False).head(15)

print(freq_df)

# Visualizar términos más frecuentes
plt.figure(figsize=(10, 6))
sns.barplot(x='frequency', y='word', data=freq_df)
plt.title(f'Top 15 Términos Más Frecuentes (BoW)', fontweight='bold')
plt.tight_layout()
plt.savefig('charts/bow_top_terms.png', dpi=300)
plt.show()

# Analizar términos clave por sentimiento
print("\n3. TÉRMINOS CLAVE POR SENTIMIENTO (Top 5 por Clase)")
print("-" * 60)

for sentiment in df_processed['Sentiment'].unique():
    # Filtrar documentos por sentimiento
    docs_sentiment = df_processed[df_processed['Sentiment'] == sentiment][TEXT_COLUMN]
    
    # Vectorizar documentos
    X_sentiment = bow_vectorizer.transform(docs_sentiment)
    
    # Calcular frecuencias
    freq_sentiment = np.asarray(X_sentiment.sum(axis=0)).flatten()
    
    # Mostrar top 5 términos
    freq_df_sentiment = pd.DataFrame({
        'word': feature_names,
        'frequency': freq_sentiment
    }).sort_values('frequency', ascending=False).head(5)
    
    print(f"\n{sentiment.upper()}:")
    print(freq_df_sentiment)

# Preparar datos finales para el modelo
print("\n4. PREPARACIÓN FINAL DE DATOS")
print("-" * 60)

# Convertir matriz dispersa a DataFrame
X_dense = X_bow.toarray()
bow_final_df = pd.DataFrame(X_dense, columns=feature_names)

# Añadir columnas de etiquetas
bow_final_df['Sentiment'] = df_processed['Sentiment'].values
bow_final_df['Idioma'] = df_processed['Idioma'].values

# Guardar
bow_final_df.to_csv('data_processed/datos_vectorizados_final.csv', index=False)
print(f"✓ Matriz de características guardada en 'data_processed/datos_vectorizados_final.csv'")

## 6. Representación Tradicional: TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

print("\n" + "=" * 60)
print("VECTORIZACIÓN: TF-IDF")
print("=" * 60)

# Seleccionar columna de texto procesado
TEXT_COLUMN = 'text_lemmatized'

# Crear vectorizador TF-IDF
print("\n1. CREANDO MATRIZ TF-IDF")
print("-" * 60)

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2),
    sublinear_tf=True
)

# Crear matriz TF-IDF
try:
    X_tfidf = tfidf_vectorizer.fit_transform(df_processed[TEXT_COLUMN])
except NameError:
    print("ERROR: El DataFrame 'df_processed' no está definido. Asegúrate de cargar los datos antes.")
    sys.exit()

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

print(f"✓ Modelo TF-IDF creado. Forma: {X_tfidf.shape}")
print(f"Tamaño del Vocabulario Final: {X_tfidf.shape[1]}")

# Analizar términos con mayor peso TF-IDF
print("\n2. TÉRMINOS CON MAYOR PESO TF-IDF PROMEDIO (TOP 15)")
print("-" * 60)

# Calcular peso promedio de cada término
tfidf_means = np.asarray(X_tfidf.mean(axis=0)).flatten()
tfidf_df = pd.DataFrame({
    'term': tfidf_feature_names,
    'tfidf_mean': tfidf_means
}).sort_values('tfidf_mean', ascending=False).head(15)

print(tfidf_df)

# Preparar datos finales
print("\n3. PREPARACIÓN FINAL DE DATOS")
print("-" * 60)

# Convertir matriz dispersa a DataFrame
tfidf_matrix_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_feature_names)
tfidf_matrix_df['Sentiment'] = df_processed['Sentiment'].values

# Guardar
tfidf_matrix_df.to_csv('data_processed/datos_tfidf_final.csv', index=False)
print(f"✓ Matriz de características guardada en 'data_processed/datos_tfidf_final.csv'")

## 7. Word Embeddings

In [ ]:
# Crear embeddings usando Word2Vec y FastText
from gensim.models import Word2Vec, FastText
import pandas as pd
import numpy as np

print("=" * 60)
print("WORD EMBEDDINGS NO CONTEXTUALES")
print("NOTA: Usamos tokens_clean (sin lematizar) porque Word2Vec/FastText")
print("      fueron entrenados con texto natural y esperan palabras en su forma original")
print("=" * 60)

# Cargar datos procesados
df_processed = pd.read_csv('data_processed/datos_preprocesados_completo.csv')

# Convertir tokens a listas
import ast

def ensure_list(value):
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except:
            return value.split()
    return []

df_processed['tokens_clean'] = df_processed['tokens_clean'].apply(ensure_list)
sentences = df_processed['tokens_clean'].tolist()

sample = sentences[0]
print(f"\nVerificación inicial:")
print(f"  Tipo: {type(sample)}")
print(f"  Ejemplo: {sample[:5] if len(sample) > 0 else 'vacío'}")

# WORD2VEC
print("\n" + "=" * 60)
print("1. WORD2VEC")
print("=" * 60)

print("\nEntrenando modelo Word2Vec...")
w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    epochs=10
)

vocab_size = len(w2v_model.wv)
print(f"✓ Modelo entrenado")
print(f"  Vocabulario: {vocab_size} palabras")
print(f"  Dimensión: {w2v_model.wv.vector_size}")

# Calcular palabras fuera de vocabulario (OOV)
oov_count = sum(1 for tokens in sentences for token in tokens if token not in w2v_model.wv)
total_tokens = sum(len(tokens) for tokens in sentences)
oov_percentage = (oov_count / total_tokens * 100) if total_tokens > 0 else 0

print(f"\nAnálisis OOV:")
print(f"  Palabras fuera de vocabulario: {oov_count:,}/{total_tokens:,}")
print(f"  Porcentaje OOV: {oov_percentage:.2f}%")

# Generar vectores de documentos promediando los vectores de palabras
def get_document_vector_w2v(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.wv.vector_size)

print("\nGenerando vectores de documentos...")
doc_vectors_w2v = df_processed['tokens_clean'].apply(
    lambda tokens: get_document_vector_w2v(tokens, w2v_model)
)

X_w2v = np.vstack(doc_vectors_w2v.values)
w2v_df = pd.DataFrame(X_w2v, columns=[f'w2v_{i}' for i in range(X_w2v.shape[1])])
w2v_df['Sentiment'] = df_processed['Sentiment'].values
w2v_df['Idioma'] = df_processed['Idioma'].values

w2v_df.to_csv('data_processed/datos_word2vec.csv', index=False)
w2v_model.save('models/word2vec_model.model')
print("✓ Vectores Word2Vec guardados")

# Ejemplo de palabras similares
if vocab_size > 1000:
    try:
        test_words = ['company', 'profit', 'loss', 'market']
        print("\nEjemplos de palabras similares:")
        for word in test_words:
            if word in w2v_model.wv:
                similar = w2v_model.wv.most_similar(word, topn=3)
                print(f"  {word}: {[w for w, s in similar]}")
                break
    except:
        pass

# FASTTEXT
print("\n" + "=" * 60)
print("2. FASTTEXT")
print("=" * 60)

print("\nEntrenando modelo FastText...")
ft_model = FastText(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    epochs=10
)

vocab_size_ft = len(ft_model.wv)
print(f"✓ Modelo entrenado")
print(f"  Vocabulario: {vocab_size_ft} palabras")
print(f"  Dimensión: {ft_model.wv.vector_size}")

print(f"\nVentaja de FastText:")
print(f"  ✓ Todas las palabras tienen representación (incluso OOV)")
print(f"  ✓ Usa subwords para palabras desconocidas")
print(f"  Total de tokens procesados: {total_tokens:,}")

# Generar vectores de documentos (FastText no tiene problema con OOV)
def get_document_vector_ft(tokens, model):
    vectors = [model.wv[word] for word in tokens]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.wv.vector_size)

print("\nGenerando vectores de documentos...")
doc_vectors_ft = df_processed['tokens_clean'].apply(
    lambda tokens: get_document_vector_ft(tokens, ft_model)
)

X_ft = np.vstack(doc_vectors_ft.values)
ft_df = pd.DataFrame(X_ft, columns=[f'ft_{i}' for i in range(X_ft.shape[1])])
ft_df['Sentiment'] = df_processed['Sentiment'].values
ft_df['Idioma'] = df_processed['Idioma'].values

ft_df.to_csv('data_processed/datos_fasttext.csv', index=False)
ft_model.save('models/fasttext_model.model')
print("✓ Vectores FastText guardados")

# Ejemplo de palabras similares
if vocab_size_ft > 1000:
    try:
        test_words = ['company', 'profit', 'loss', 'market']
        print("\nEjemplos de palabras similares:")
        for word in test_words:
            similar = ft_model.wv.most_similar(word, topn=3)
            print(f"  {word}: {[w for w, s in similar]}")
            break
    except:
        pass

print("\n" + "=" * 60)
print("RESUMEN DE EMBEDDINGS NO CONTEXTUALES")
print("=" * 60)
print(f"✓ Word2Vec: {vocab_size:,} palabras, {oov_percentage:.2f}% OOV")
print(f"✓ FastText: {vocab_size_ft:,} palabras, 0% OOV (usa subwords)")

## 8. Word Embeddings Contextuales con BERT

In [ ]:
# Crear embeddings contextuales usando BERT
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

print("=" * 60)
print("WORD EMBEDDINGS CONTEXTUALES - BERT")
print("=" * 60)

# Cargar datos
df_processed = pd.read_csv('data_processed/datos_preprocesados_completo.csv')

# Cargar modelo BERT multilingüe
model_name = 'bert-base-multilingual-cased'
print(f"\nCargando modelo: {model_name}")
print("(Este proceso puede tardar unos minutos la primera vez)")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Usar GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

print(f"✓ Modelo cargado en: {device}")
print(f"  Vocabulario: {len(tokenizer)} tokens")
print(f"  Dimensión de embeddings: {model.config.hidden_size}")

# Obtener embedding de BERT usando el token [CLS]
def get_bert_embedding(text, tokenizer, model, device, max_length=128):
    # Tokenizar texto
    inputs = tokenizer(text, return_tensors='pt', truncation=True, 
                      padding=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Obtener embeddings sin calcular gradientes
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Usar el embedding del token [CLS] que representa toda la oración
    cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embedding.flatten()

# Generar embeddings para todos los documentos
print(f"\nGenerando embeddings para {len(df_processed)} documentos...")
print("(Esto puede tardar varios minutos)")

embeddings_list = []
batch_size = 32

for i in tqdm(range(0, len(df_processed), batch_size)):
    batch = df_processed['Sentence'].iloc[i:i+batch_size].tolist()
    
    for text in batch:
        embedding = get_bert_embedding(str(text), tokenizer, model, device)
        embeddings_list.append(embedding)

# Crear DataFrame con embeddings
X_bert = np.vstack(embeddings_list)
bert_df = pd.DataFrame(X_bert, columns=[f'bert_{i}' for i in range(X_bert.shape[1])])
bert_df['Sentiment'] = df_processed['Sentiment'].values
bert_df['Idioma'] = df_processed['Idioma'].values

# Guardar
bert_df.to_csv('data_processed/datos_bert.csv', index=False)
print(f"\n✓ Embeddings BERT guardados")
print(f"  Forma de la matriz: {X_bert.shape}")
print(f"  Archivo: datos_bert.csv")

print(f"\nAnálisis OOV:")
print(f"  ✓ BERT no tiene palabras OOV")
print(f"  ✓ Usa subword tokenization (WordPiece)")
print(f"  ✓ Embeddings contextuales (varía según contexto)")

print("\n" + "=" * 60)
print("COMPARACIÓN: NO CONTEXTUALES vs CONTEXTUALES")
print("=" * 60)
print("\nWord2Vec/FastText (No Contextuales):")
print("  • Cada palabra tiene UN SOLO vector")
print("  • No considera contexto")
print("  • Vocabulario limitado (problemas OOV)")
print("\nBERT (Contextual):")
print("  • Cada palabra tiene MÚLTIPLES vectores según contexto")
print("  • Considera contexto completo de la oración")
print("  • Sin problemas OOV (subword tokenization)")
print("  • Más costoso computacionalmente")

## 10. TAREA 1: División Train/Validation/Test

**Objetivo**: Crear splits estratificados para ambas tareas (Consistencia y Sentimiento)

En esta sección vamos a dividir el dataset en conjuntos de entrenamiento, validación y prueba de forma estratificada para garantizar que las proporciones de clases se mantengan en cada split.

**Splits a crear**:
- Train: 70%
- Validation: 15%
- Test: 15%

**Tareas**:
1. Detección de Consistencia (target: Etiqueta - correcta/incorrecta)
2. Análisis de Sentimiento (target: Sentiment - positive/negative/neutral)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Establecer semilla para reproducibilidad
np.random.seed(42)

print("=" * 80)
print("TAREA 1: DIVISIÓN DE DATOS EN TRAIN/VALIDATION/TEST")
print("=" * 80)

# Cargar dataset multiidioma
print("\n1. CARGANDO DATASET PRINCIPAL")
print("-" * 80)

df_main = pd.read_csv('data/initial_data.csv')
print(f"✓ Dataset cargado: {len(df_main)} noticias")
print(f"\nColumnas disponibles: {list(df_main.columns)}")

# Verificar si existe la columna 'Etiqueta', si no, crearla
if 'Etiqueta' not in df_main.columns:
    print("\nNOTA: Columna 'Etiqueta' no encontrada. Creando todas las noticias como 'correcta'")
    df_main['Etiqueta'] = 'correcta'

# Verificar distribuciones
print(f"\nDistribución de Sentiment:")
print(df_main['Sentiment'].value_counts())
print(f"\nDistribución de Etiqueta:")
print(df_main['Etiqueta'].value_counts())

# DIVISIÓN PARA TAREA 1: DETECCIÓN DE CONSISTENCIA (Etiqueta: correcta/incorrecta)
print("\n" + "=" * 80)
print("DIVISIÓN PARA TAREA 1: DETECCIÓN DE CONSISTENCIA")
print("=" * 80)

# Primero dividir en train (70%) y temp (30%)
X_consistency = df_main['Sentence'].values
y_consistency = df_main['Etiqueta'].values

X_train_cons, X_temp_cons, y_train_cons, y_temp_cons = train_test_split(
    X_consistency, y_consistency,
    test_size=0.30,
    random_state=42,
    stratify=y_consistency
)

# Luego dividir temp en validation (15%) y test (15%)
X_val_cons, X_test_cons, y_val_cons, y_test_cons = train_test_split(
    X_temp_cons, y_temp_cons,
    test_size=0.50,
    random_state=42,
    stratify=y_temp_cons
)

# Crear DataFrames
consistency_train = pd.DataFrame({
    'Sentence': X_train_cons,
    'Etiqueta': y_train_cons
})
consistency_val = pd.DataFrame({
    'Sentence': X_val_cons,
    'Etiqueta': y_val_cons
})
consistency_test = pd.DataFrame({
    'Sentence': X_test_cons,
    'Etiqueta': y_test_cons
})

print(f"\n✓ Splits creados para Detección de Consistencia:")
print(f"  Train: {len(consistency_train)} ({len(consistency_train)/len(df_main)*100:.1f}%)")
print(f"  Validation: {len(consistency_val)} ({len(consistency_val)/len(df_main)*100:.1f}%)")
print(f"  Test: {len(consistency_test)} ({len(consistency_test)/len(df_main)*100:.1f}%)")

# Verificar balanceo en cada split
print(f"\nDistribución de clases en cada split:")
print(f"\nTrain:")
print(consistency_train['Etiqueta'].value_counts(normalize=True).mul(100).round(2))
print(f"\nValidation:")
print(consistency_val['Etiqueta'].value_counts(normalize=True).mul(100).round(2))
print(f"\nTest:")
print(consistency_test['Etiqueta'].value_counts(normalize=True).mul(100).round(2))

# DIVISIÓN PARA TAREA 2: ANÁLISIS DE SENTIMIENTO (Sentiment: positive/negative/neutral)
print("\n" + "=" * 80)
print("DIVISIÓN PARA TAREA 2: ANÁLISIS DE SENTIMIENTO")
print("=" * 80)

# Primero dividir en train (70%) y temp (30%)
X_sentiment = df_main['Sentence'].values
y_sentiment = df_main['Sentiment'].values

X_train_sent, X_temp_sent, y_train_sent, y_temp_sent = train_test_split(
    X_sentiment, y_sentiment,
    test_size=0.30,
    random_state=42,
    stratify=y_sentiment
)

# Luego dividir temp en validation (15%) y test (15%)
X_val_sent, X_test_sent, y_val_sent, y_test_sent = train_test_split(
    X_temp_sent, y_temp_sent,
    test_size=0.50,
    random_state=42,
    stratify=y_temp_sent
)

# Crear DataFrames
sentiment_train = pd.DataFrame({
    'Sentence': X_train_sent,
    'Sentiment': y_train_sent
})
sentiment_val = pd.DataFrame({
    'Sentence': X_val_sent,
    'Sentiment': y_val_sent
})
sentiment_test = pd.DataFrame({
    'Sentence': X_test_sent,
    'Sentiment': y_test_sent
})

print(f"\n✓ Splits creados para Análisis de Sentimiento:")
print(f"  Train: {len(sentiment_train)} ({len(sentiment_train)/len(df_main)*100:.1f}%)")
print(f"  Validation: {len(sentiment_val)} ({len(sentiment_val)/len(df_main)*100:.1f}%)")
print(f"  Test: {len(sentiment_test)} ({len(sentiment_test)/len(df_main)*100:.1f}%)")

# Verificar balanceo en cada split
print(f"\nDistribución de clases en cada split:")
print(f"\nTrain:")
print(sentiment_train['Sentiment'].value_counts(normalize=True).mul(100).round(2))
print(f"\nValidation:")
print(sentiment_val['Sentiment'].value_counts(normalize=True).mul(100).round(2))
print(f"\nTest:")
print(sentiment_test['Sentiment'].value_counts(normalize=True).mul(100).round(2))

# GUARDAR SPLITS EN ARCHIVOS CSV
print("\n" + "=" * 80)
print("GUARDANDO SPLITS EN ARCHIVOS CSV")
print("=" * 80)

# Guardar splits de consistencia
consistency_train.to_csv('data_processed/consistency_train.csv', index=False)
consistency_val.to_csv('data_processed/consistency_val.csv', index=False)
consistency_test.to_csv('data_processed/consistency_test.csv', index=False)

print(f"\n✓ Splits de Consistencia guardados:")
print(f"  data_processed/consistency_train.csv")
print(f"  data_processed/consistency_val.csv")
print(f"  data_processed/consistency_test.csv")

# Guardar splits de sentimiento
sentiment_train.to_csv('data_processed/sentiment_train.csv', index=False)
sentiment_val.to_csv('data_processed/sentiment_val.csv', index=False)
sentiment_test.to_csv('data_processed/sentiment_test.csv', index=False)

print(f"\n✓ Splits de Sentimiento guardados:")
print(f"  data_processed/sentiment_train.csv")
print(f"  data_processed/sentiment_val.csv")
print(f"  data_processed/sentiment_test.csv")

# VISUALIZACIÓN DE LA DISTRIBUCIÓN
print("\n" + "=" * 80)
print("VISUALIZACIÓN DE LA DISTRIBUCIÓN DE SPLITS")
print("=" * 80)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Distribución de Clases en Train/Val/Test', fontsize=16, fontweight='bold')

# Fila 1: Consistencia
splits_cons = [
    (consistency_train, 'Train (Consistencia)'),
    (consistency_val, 'Validation (Consistencia)'),
    (consistency_test, 'Test (Consistencia)')
]

for idx, (split_df, title) in enumerate(splits_cons):
    counts = split_df['Etiqueta'].value_counts()
    colors = ['#2ecc71' if label == 'correcta' else '#e74c3c' for label in counts.index]
    axes[0, idx].bar(counts.index, counts.values, color=colors)
    axes[0, idx].set_title(title)
    axes[0, idx].set_ylabel('Frecuencia')
    axes[0, idx].set_xlabel('Etiqueta')

    # Añadir porcentajes
    for i, (label, count) in enumerate(counts.items()):
        pct = count / len(split_df) * 100
        axes[0, idx].text(i, count, f'{pct:.1f}%', ha='center', va='bottom')

# Fila 2: Sentimiento
splits_sent = [
    (sentiment_train, 'Train (Sentimiento)'),
    (sentiment_val, 'Validation (Sentimiento)'),
    (sentiment_test, 'Test (Sentimiento)')
]

for idx, (split_df, title) in enumerate(splits_sent):
    counts = split_df['Sentiment'].value_counts()
    axes[1, idx].bar(range(len(counts)), counts.values)
    axes[1, idx].set_title(title)
    axes[1, idx].set_ylabel('Frecuencia')
    axes[1, idx].set_xlabel('Sentimiento')
    axes[1, idx].set_xticks(range(len(counts)))
    axes[1, idx].set_xticklabels(counts.index, rotation=45)

    # Añadir porcentajes
    for i, (label, count) in enumerate(counts.items()):
        pct = count / len(split_df) * 100
        axes[1, idx].text(i, count, f'{pct:.1f}%', ha='center', va='bottom')

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig('charts/04_train_val_test_splits.png', dpi=300, bbox_inches='tight')
print("\n✓ Visualización guardada: charts/04_train_val_test_splits.png")
plt.show()

# RESUMEN FINAL
print("\n" + "=" * 80)
print("RESUMEN FINAL")
print("=" * 80)

print(f"\n✓ TAREA 1 COMPLETADA")
print(f"\nArchivos generados:")
print(f"  1. Splits para Detección de Consistencia (3 archivos)")
print(f"  2. Splits para Análisis de Sentimiento (3 archivos)")
print(f"  3. Visualización de distribuciones (1 gráfico)")
print(f"\nTotal de splits creados: 6 archivos CSV")
print(f"Proporción: 70% Train, 15% Validation, 15% Test")
print(f"Stratificación: ✓ Aplicada en ambas tareas")
print(f"\nLos splits están listos para ser usados en las siguientes tareas.")

## 11. TAREA 2: Shallow Learning - Detección de Consistencia con BoW

**Objetivo**: Entrenar y comparar 3 clasificadores tradicionales para detectar consistencia usando representación Bag of Words.

En esta sección vamos a:
1. Cargar los datos vectorizados con BoW (ya generados en la sección 5)
2. Aplicar los splits Train/Val/Test de la TAREA 1
3. Entrenar 3 clasificadores: Logistic Regression, Random Forest, SVM
4. Optimizar hiperparámetros con GridSearchCV
5. Evaluar y comparar rendimiento

**Tarea**: Detección de Consistencia (correcta vs incorrecta)  
**Representación**: Bag of Words (BoW)  
**Clasificadores**: Logistic Regression, Random Forest, LinearSVC

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

# Establecer semilla
np.random.seed(42)

print("=" * 80)
print("TAREA 2: SHALLOW LEARNING - DETECCIÓN DE CONSISTENCIA CON BoW")
print("=" * 80)

# PASO 1: CARGAR DATOS BoW Y SPLITS
print("\n1. CARGANDO DATOS BoW")
print("-" * 80)

# Cargar datos BoW completos (generados en sección 5)
# NOTA: Si no existe datos_vectorizados_final.csv, necesitamos generarlo
try:
    df_bow = pd.read_csv('data_processed/datos_vectorizados_final.csv')
    print(f"✓ Datos BoW cargados: {df_bow.shape}")
    print(f"  Características: {df_bow.shape[1] - 2}")  # -2 por Sentiment e Idioma
except FileNotFoundError:
    print("ERROR: No se encontró 'datos_vectorizados_final.csv'")
    print("Por favor, ejecuta primero la sección 5 (Bag of Words)")
    raise

# Cargar splits de consistencia
print("\n2. CARGANDO SPLITS DE CONSISTENCIA")
print("-" * 80)

try:
    consistency_train = pd.read_csv('data_processed/consistency_train.csv')
    consistency_val = pd.read_csv('data_processed/consistency_val.csv')
    consistency_test = pd.read_csv('data_processed/consistency_test.csv')
    
    print(f"✓ Splits cargados:")
    print(f"  Train: {len(consistency_train)} muestras")
    print(f"  Validation: {len(consistency_val)} muestras")
    print(f"  Test: {len(consistency_test)} muestras")
except FileNotFoundError:
    print("ERROR: No se encontraron los splits de consistencia")
    print("Por favor, ejecuta primero la TAREA 1 (División Train/Val/Test)")
    raise

# Verificar que tenemos la columna Etiqueta en los datos BoW
# Necesitamos fusionar los datos BoW con las etiquetas de consistencia
print("\n3. PREPARANDO DATOS PARA ENTRENAMIENTO")
print("-" * 80)

# Cargar el dataset original con etiquetas
df_original = pd.read_csv('data/initial_data.csv')

# Verificar/crear columna Etiqueta
if 'Etiqueta' not in df_original.columns:
    print("NOTA: Creando columna 'Etiqueta' = 'correcta' (todas correctas)")
    df_original['Etiqueta'] = 'correcta'

# Ahora necesitamos hacer match entre los datos BoW y las etiquetas
# Asumimos que el orden es el mismo
print(f"Verificando concordancia de tamaños...")
print(f"  BoW shape: {df_bow.shape[0]}")
print(f"  Original shape: {df_original.shape[0]}")

# Agregar columna Etiqueta a df_bow
if df_bow.shape[0] == df_original.shape[0]:
    df_bow['Etiqueta'] = df_original['Etiqueta'].values
    print("✓ Etiquetas agregadas a datos BoW")
else:
    print("⚠️  ADVERTENCIA: Tamaños no coinciden. Esto puede causar problemas.")

# Preparar conjuntos de entrenamiento, validación y test
# Necesitamos hacer merge con las oraciones de los splits

def prepare_bow_data(split_df, df_bow_full):
    """Prepara datos BoW para un split específico"""
    # Crear una copia del split con índice basado en Sentence
    split_sentences = split_df['Sentence'].values
    
    # Encontrar índices correspondientes en df_original
    df_original_indexed = df_original.reset_index(drop=True)
    indices = []
    
    for sentence in split_sentences:
        # Buscar la oración en el dataset original
        matches = df_original_indexed[df_original_indexed['Sentence'] == sentence].index
        if len(matches) > 0:
            indices.append(matches[0])
    
    # Extraer características BoW correspondientes
    X = df_bow_full.iloc[indices].drop(['Sentiment', 'Idioma', 'Etiqueta'], axis=1, errors='ignore').values
    y = split_df['Etiqueta'].values
    
    return X, y

print("\nPreparando splits con características BoW...")
X_train, y_train = prepare_bow_data(consistency_train, df_bow)
X_val, y_val = prepare_bow_data(consistency_val, df_bow)
X_test, y_test = prepare_bow_data(consistency_test, df_bow)

print(f"✓ Datos preparados:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"  X_test: {X_test.shape}, y_test: {y_test.shape}")

# PASO 2: ENTRENAR CLASIFICADORES CON GRIDSEARCH
print("\n" + "=" * 80)
print("4. ENTRENAMIENTO Y OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 80)

# Definir modelos y grids de hiperparámetros
models = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000, random_state=42),
        'params': {
            'C': [0.1, 1, 10]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, None]
        }
    },
    'LinearSVC': {
        'model': LinearSVC(random_state=42, max_iter=2000),
        'params': {
            'C': [0.1, 1, 10]
        }
    }
}

# Entrenar y evaluar cada modelo
results = []
best_models = {}

for model_name, config in models.items():
    print(f"\n--- {model_name} ---")
    print(f"Hiperparámetros a probar: {config['params']}")
    
    # GridSearchCV
    start_time = time.time()
    grid_search = GridSearchCV(
        config['model'],
        config['params'],
        cv=5,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Mejor modelo
    best_model = grid_search.best_estimator_
    best_models[model_name] = best_model
    
    print(f"✓ Entrenamiento completado en {training_time:.2f}s")
    print(f"Mejores hiperparámetros: {grid_search.best_params_}")
    
    # Evaluar en validation set
    y_val_pred = best_model.predict(X_val)
    
    # Calcular métricas
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average='weighted', zero_division=0)
    recall = recall_score(y_val, y_val_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_val, y_val_pred, average='weighted', zero_division=0)
    
    results.append({
        'Modelo': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Tiempo_Entrenamiento': training_time,
        'Mejores_Params': str(grid_search.best_params_)
    })
    
    print(f"Métricas en Validation:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")

# PASO 3: COMPARAR RESULTADOS
print("\n" + "=" * 80)
print("5. COMPARACIÓN DE RESULTADOS")
print("=" * 80)

results_df = pd.DataFrame(results)
print("\nTabla comparativa:")
print(results_df.to_string(index=False))

# Identificar mejor modelo
best_model_name = results_df.loc[results_df['F1-Score'].idxmax(), 'Modelo']
print(f"\n✓ Mejor modelo según F1-Score: {best_model_name}")

# PASO 4: EVALUACIÓN FINAL EN TEST SET
print("\n" + "=" * 80)
print("6. EVALUACIÓN FINAL EN TEST SET")
print("=" * 80)

best_model_final = best_models[best_model_name]
y_test_pred = best_model_final.predict(X_test)

print(f"\nResultados del mejor modelo ({best_model_name}) en Test:")
print(classification_report(y_test, y_test_pred))

# PASO 5: MATRICES DE CONFUSIÓN
print("\n" + "=" * 80)
print("7. MATRICES DE CONFUSIÓN")
print("=" * 80)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Matrices de Confusión - BoW (Validation Set)', fontsize=16, fontweight='bold')

for idx, (model_name, model) in enumerate(best_models.items()):
    y_val_pred = model.predict(X_val)
    cm = confusion_matrix(y_val, y_val_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['correcta', 'incorrecta'],
                yticklabels=['correcta', 'incorrecta'])
    axes[idx].set_title(f'{model_name}\nF1: {results_df[results_df["Modelo"]==model_name]["F1-Score"].values[0]:.4f}')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig('charts/05_bow_consistency_confusion_matrices.png', dpi=300, bbox_inches='tight')
print("✓ Matrices de confusión guardadas: charts/05_bow_consistency_confusion_matrices.png")
plt.show()

# PASO 6: GRÁFICO COMPARATIVO
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Comparación de Modelos - BoW (Consistencia)', fontsize=16, fontweight='bold')

x_pos = np.arange(len(results_df))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
width = 0.2

for i, metric in enumerate(metrics):
    ax.bar(x_pos + i*width, results_df[metric], width, label=metric)

ax.set_ylabel('Score')
ax.set_xlabel('Modelo')
ax.set_xticks(x_pos + width * 1.5)
ax.set_xticklabels(results_df['Modelo'], rotation=15, ha='right')
ax.legend()
ax.set_ylim([0, 1.1])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig('charts/05_bow_consistency_comparison.png', dpi=300, bbox_inches='tight')
print("✓ Gráfico comparativo guardado: charts/05_bow_consistency_comparison.png")
plt.show()

# PASO 7: GUARDAR MEJOR MODELO
print("\n" + "=" * 80)
print("8. GUARDANDO MEJOR MODELO")
print("=" * 80)

model_path = 'models/bow_consistency_best.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(best_model_final, f)
print(f"✓ Mejor modelo guardado: {model_path}")

# Guardar tabla de resultados
results_df.to_csv('models/bow_consistency_results.csv', index=False)
print(f"✓ Resultados guardados: models/bow_consistency_results.csv")

# RESUMEN FINAL
print("\n" + "=" * 80)
print("RESUMEN FINAL - TAREA 2")
print("=" * 80)

print(f"\n✓ TAREA 2 COMPLETADA")
print(f"\nMejor modelo: {best_model_name}")
print(f"F1-Score (validation): {results_df[results_df['Modelo']==best_model_name]['F1-Score'].values[0]:.4f}")
print(f"\nArchivos generados:")
print(f"  1. Matrices de confusión (3 modelos)")
print(f"  2. Gráfico comparativo de métricas")
print(f"  3. Mejor modelo guardado (pickle)")
print(f"  4. Tabla de resultados (CSV)")
print(f"\nJustificación de hiperparámetros:")
print(f"  - Logistic Regression: C controla la regularización (mayor C = menos regularización)")
print(f"  - Random Forest: n_estimators (número de árboles), max_depth (profundidad máxima)")
print(f"  - LinearSVC: C controla el trade-off entre margen y error de clasificación")
print(f"\nLos hiperparámetros fueron optimizados mediante GridSearchCV con validación cruzada 5-fold.")

## 12. TAREA 3: Shallow Learning - Análisis de Sentimiento con TF-IDF

**Objetivo**: Entrenar y comparar 3 clasificadores para análisis de sentimiento multiclase usando representación TF-IDF.

En esta sección vamos a:
1. Cargar los datos vectorizados con TF-IDF (ya generados en la sección 6)
2. Aplicar los splits Train/Val/Test de sentimiento de la TAREA 1
3. Entrenar 3 clasificadores: Logistic Regression (multinomial), Random Forest, Multinomial Naive Bayes
4. Optimizar hiperparámetros con GridSearchCV
5. Evaluar con métricas macro/micro/weighted (3 clases)
6. Análisis especial de la clase minoritaria "negative"

**Tarea**: Análisis de Sentimiento (positive/negative/neutral)  
**Representación**: TF-IDF  
**Clasificadores**: Logistic Regression, Random Forest, Multinomial Naive Bayes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

# Establecer semilla
np.random.seed(42)

print("=" * 80)
print("TAREA 3: SHALLOW LEARNING - ANÁLISIS DE SENTIMIENTO CON TF-IDF")
print("=" * 80)

# PASO 1: CARGAR DATOS TF-IDF Y SPLITS
print("\n1. CARGANDO DATOS TF-IDF")
print("-" * 80)

try:
    df_tfidf = pd.read_csv('data_processed/datos_tfidf_final.csv')
    print(f"✓ Datos TF-IDF cargados: {df_tfidf.shape}")
    print(f"  Características: {df_tfidf.shape[1] - 1}")  # -1 por Sentiment
except FileNotFoundError:
    print("ERROR: No se encontró 'datos_tfidf_final.csv'")
    print("Por favor, ejecuta primero la sección 6 (TF-IDF)")
    raise

# Cargar splits de sentimiento
print("\n2. CARGANDO SPLITS DE SENTIMIENTO")
print("-" * 80)

try:
    sentiment_train = pd.read_csv('data_processed/sentiment_train.csv')
    sentiment_val = pd.read_csv('data_processed/sentiment_val.csv')
    sentiment_test = pd.read_csv('data_processed/sentiment_test.csv')
    
    print(f"✓ Splits cargados:")
    print(f"  Train: {len(sentiment_train)} muestras")
    print(f"  Validation: {len(sentiment_val)} muestras")
    print(f"  Test: {len(sentiment_test)} muestras")
    
    # Mostrar distribución de clases
    print(f"\nDistribución de clases en Train:")
    print(sentiment_train['Sentiment'].value_counts())
    print(f"\nDistribución de clases en Validation:")
    print(sentiment_val['Sentiment'].value_counts())
    print(f"\nDistribución de clases en Test:")
    print(sentiment_test['Sentiment'].value_counts())
    
except FileNotFoundError:
    print("ERROR: No se encontraron los splits de sentimiento")
    print("Por favor, ejecuta primero la TAREA 1 (División Train/Val/Test)")
    raise

# PASO 2: PREPARAR DATOS
print("\n3. PREPARANDO DATOS PARA ENTRENAMIENTO")
print("-" * 80)

# Cargar dataset original
df_original = pd.read_csv('data/initial_data.csv')

def prepare_tfidf_data(split_df, df_tfidf_full):
    """Prepara datos TF-IDF para un split específico"""
    split_sentences = split_df['Sentence'].values
    df_original_indexed = df_original.reset_index(drop=True)
    indices = []
    
    for sentence in split_sentences:
        matches = df_original_indexed[df_original_indexed['Sentence'] == sentence].index
        if len(matches) > 0:
            indices.append(matches[0])
    
    # Extraer características TF-IDF correspondientes
    X = df_tfidf_full.iloc[indices].drop(['Sentiment'], axis=1, errors='ignore').values
    y = split_df['Sentiment'].values
    
    return X, y

print("Preparando splits con características TF-IDF...")
X_train, y_train = prepare_tfidf_data(sentiment_train, df_tfidf)
X_val, y_val = prepare_tfidf_data(sentiment_val, df_tfidf)
X_test, y_test = prepare_tfidf_data(sentiment_test, df_tfidf)

print(f"✓ Datos preparados:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"  X_test: {X_test.shape}, y_test: {y_test.shape}")

# Análisis de clases
print(f"\nAnálisis de distribución de clases:")
unique, counts = np.unique(y_train, return_counts=True)
for label, count in zip(unique, counts):
    pct = count / len(y_train) * 100
    print(f"  {label}: {count} ({pct:.2f}%)")

# PASO 3: ENTRENAR CLASIFICADORES CON GRIDSEARCH
print("\n" + "=" * 80)
print("4. ENTRENAMIENTO Y OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("=" * 80)

# Definir modelos y grids de hiperparámetros
models = {
    'Logistic Regression': {
        'model': LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42),
        'params': {
            'C': [0.1, 1, 10],
            'solver': ['lbfgs', 'saga']
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [15, 25, None]
        }
    },
    'Multinomial NB': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.1, 0.5, 1.0]
        }
    }
}

# Entrenar y evaluar cada modelo
results = []
best_models = {}

for model_name, config in models.items():
    print(f"\n--- {model_name} ---")
    print(f"Hiperparámetros a probar: {config['params']}")
    
    # GridSearchCV
    start_time = time.time()
    grid_search = GridSearchCV(
        config['model'],
        config['params'],
        cv=5,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Mejor modelo
    best_model = grid_search.best_estimator_
    best_models[model_name] = best_model
    
    print(f"✓ Entrenamiento completado en {training_time:.2f}s")
    print(f"Mejores hiperparámetros: {grid_search.best_params_}")
    
    # Evaluar en validation set
    y_val_pred = best_model.predict(X_val)
    
    # Calcular métricas con diferentes promedios
    accuracy = accuracy_score(y_val, y_val_pred)
    precision_macro = precision_score(y_val, y_val_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_val, y_val_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_val, y_val_pred, average='macro', zero_division=0)
    
    precision_micro = precision_score(y_val, y_val_pred, average='micro', zero_division=0)
    recall_micro = recall_score(y_val, y_val_pred, average='micro', zero_division=0)
    f1_micro = f1_score(y_val, y_val_pred, average='micro', zero_division=0)
    
    precision_weighted = precision_score(y_val, y_val_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_val, y_val_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_val, y_val_pred, average='weighted', zero_division=0)
    
    results.append({
        'Modelo': model_name,
        'Accuracy': accuracy,
        'F1_Macro': f1_macro,
        'F1_Micro': f1_micro,
        'F1_Weighted': f1_weighted,
        'Precision_Macro': precision_macro,
        'Recall_Macro': recall_macro,
        'Tiempo_Entrenamiento': training_time,
        'Mejores_Params': str(grid_search.best_params_)
    })
    
    print(f"Métricas en Validation:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  F1-Macro: {f1_macro:.4f}")
    print(f"  F1-Micro: {f1_micro:.4f}")
    print(f"  F1-Weighted: {f1_weighted:.4f}")

# PASO 4: COMPARAR RESULTADOS
print("\n" + "=" * 80)
print("5. COMPARACIÓN DE RESULTADOS")
print("=" * 80)

results_df = pd.DataFrame(results)
print("\nTabla comparativa:")
print(results_df[['Modelo', 'Accuracy', 'F1_Macro', 'F1_Micro', 'F1_Weighted', 'Tiempo_Entrenamiento']].to_string(index=False))

# Identificar mejor modelo
best_model_name = results_df.loc[results_df['F1_Weighted'].idxmax(), 'Modelo']
print(f"\n✓ Mejor modelo según F1-Weighted: {best_model_name}")

# PASO 5: EVALUACIÓN DETALLADA POR CLASE
print("\n" + "=" * 80)
print("6. EVALUACIÓN DETALLADA POR CLASE (Validation Set)")
print("=" * 80)

for model_name, model in best_models.items():
    print(f"\n--- {model_name} ---")
    y_val_pred = model.predict(X_val)
    print(classification_report(y_val, y_val_pred, zero_division=0))

# PASO 6: ANÁLISIS DE CLASE MINORITARIA
print("\n" + "=" * 80)
print("7. ANÁLISIS ESPECIAL DE CLASE MINORITARIA (negative)")
print("=" * 80)

best_model_final = best_models[best_model_name]
y_val_pred = best_model_final.predict(X_val)

# Obtener métricas por clase
report_dict = classification_report(y_val, y_val_pred, output_dict=True, zero_division=0)

print(f"\nMétricas del mejor modelo ({best_model_name}) para cada clase:")
print(f"\n{'Clase':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
print("-" * 65)

for label in ['negative', 'neutral', 'positive']:
    if label in report_dict:
        metrics = report_dict[label]
        print(f"{label:<15} {metrics['precision']:<12.4f} {metrics['recall']:<12.4f} {metrics['f1-score']:<12.4f} {int(metrics['support']):<10}")

# Analizar errores en clase "negative"
print(f"\nAnálisis de errores en clase 'negative':")
negative_indices = np.where(y_val == 'negative')[0]
negative_predictions = y_val_pred[negative_indices]
negative_true = y_val[negative_indices]

errors = np.sum(negative_predictions != negative_true)
total = len(negative_true)
accuracy_negative = 1 - (errors / total)

print(f"  Total de muestras 'negative': {total}")
print(f"  Correctamente clasificadas: {total - errors}")
print(f"  Incorrectamente clasificadas: {errors}")
print(f"  Accuracy en clase 'negative': {accuracy_negative:.4f}")

# Ver a qué clases se confunden las 'negative'
if errors > 0:
    print(f"\nConfusión de clase 'negative':")
    unique_preds, counts = np.unique(negative_predictions[negative_predictions != negative_true], return_counts=True)
    for pred_class, count in zip(unique_preds, counts):
        print(f"  Clasificadas como '{pred_class}': {count} ({count/errors*100:.1f}% de los errores)")

# PASO 7: EVALUACIÓN FINAL EN TEST SET
print("\n" + "=" * 80)
print("8. EVALUACIÓN FINAL EN TEST SET")
print("=" * 80)

y_test_pred = best_model_final.predict(X_test)

print(f"\nResultados del mejor modelo ({best_model_name}) en Test:")
print(classification_report(y_test, y_test_pred, zero_division=0))

# PASO 8: MATRICES DE CONFUSIÓN
print("\n" + "=" * 80)
print("9. MATRICES DE CONFUSIÓN")
print("=" * 80)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Matrices de Confusión - TF-IDF (Validation Set)', fontsize=16, fontweight='bold')

labels_order = ['negative', 'neutral', 'positive']

for idx, (model_name, model) in enumerate(best_models.items()):
    y_val_pred = model.predict(X_val)
    cm = confusion_matrix(y_val, y_val_pred, labels=labels_order)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrRd', ax=axes[idx],
                xticklabels=labels_order,
                yticklabels=labels_order)
    axes[idx].set_title(f'{model_name}\nF1-Weighted: {results_df[results_df["Modelo"]==model_name]["F1_Weighted"].values[0]:.4f}')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig('charts/06_tfidf_sentiment_confusion_matrices.png', dpi=300, bbox_inches='tight')
print("✓ Matrices de confusión guardadas: charts/06_tfidf_sentiment_confusion_matrices.png")
plt.show()

# PASO 9: GRÁFICO COMPARATIVO
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Comparación de Modelos - TF-IDF (Sentimiento)', fontsize=16, fontweight='bold')

# Gráfico 1: Métricas generales
x_pos = np.arange(len(results_df))
metrics = ['Accuracy', 'F1_Macro', 'F1_Micro', 'F1_Weighted']
width = 0.2

for i, metric in enumerate(metrics):
    axes[0].bar(x_pos + i*width, results_df[metric], width, label=metric)

axes[0].set_ylabel('Score')
axes[0].set_xlabel('Modelo')
axes[0].set_xticks(x_pos + width * 1.5)
axes[0].set_xticklabels(results_df['Modelo'], rotation=15, ha='right')
axes[0].legend()
axes[0].set_ylim([0, 1.1])
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_title('Métricas Generales')

# Gráfico 2: F1-Score por clase (mejor modelo)
classes = ['negative', 'neutral', 'positive']
f1_scores = []

for class_label in classes:
    if class_label in report_dict:
        f1_scores.append(report_dict[class_label]['f1-score'])
    else:
        f1_scores.append(0)

colors = ['#e74c3c', '#95a5a6', '#2ecc71']
axes[1].bar(classes, f1_scores, color=colors)
axes[1].set_ylabel('F1-Score')
axes[1].set_xlabel('Clase')
axes[1].set_ylim([0, 1.1])
axes[1].grid(axis='y', alpha=0.3)
axes[1].set_title(f'F1-Score por Clase ({best_model_name})')

# Añadir valores sobre las barras
for i, (label, score) in enumerate(zip(classes, f1_scores)):
    axes[1].text(i, score + 0.02, f'{score:.3f}', ha='center', va='bottom')

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig('charts/06_tfidf_sentiment_comparison.png', dpi=300, bbox_inches='tight')
print("✓ Gráfico comparativo guardado: charts/06_tfidf_sentiment_comparison.png")
plt.show()

# PASO 10: GUARDAR MEJOR MODELO
print("\n" + "=" * 80)
print("10. GUARDANDO MEJOR MODELO")
print("=" * 80)

model_path = 'models/tfidf_sentiment_best.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(best_model_final, f)
print(f"✓ Mejor modelo guardado: {model_path}")

# Guardar tabla de resultados
results_df.to_csv('models/tfidf_sentiment_results.csv', index=False)
print(f"✓ Resultados guardados: models/tfidf_sentiment_results.csv")

# RESUMEN FINAL
print("\n" + "=" * 80)
print("RESUMEN FINAL - TAREA 3")
print("=" * 80)

print(f"\n✓ TAREA 3 COMPLETADA")
print(f"\nMejor modelo: {best_model_name}")
print(f"F1-Weighted (validation): {results_df[results_df['Modelo']==best_model_name]['F1_Weighted'].values[0]:.4f}")
print(f"F1-Macro (validation): {results_df[results_df['Modelo']==best_model_name]['F1_Macro'].values[0]:.4f}")
print(f"\nRendimiento por clase (mejor modelo):")
for label in classes:
    if label in report_dict:
        print(f"  {label}: F1={report_dict[label]['f1-score']:.4f}, Support={int(report_dict[label]['support'])}")

print(f"\nArchivos generados:")
print(f"  1. Matrices de confusión (3 modelos)")
print(f"  2. Gráficos comparativos (métricas generales y por clase)")
print(f"  3. Mejor modelo guardado (pickle)")
print(f"  4. Tabla de resultados (CSV)")

print(f"\nJustificación de hiperparámetros:")
print(f"  - Logistic Regression: C (regularización), solver (optimizador para multinomial)")
print(f"  - Random Forest: n_estimators, max_depth (mayor profundidad para capturar complejidad multiclase)")
print(f"  - Multinomial NB: alpha (suavizado de Laplace, crucial para TF-IDF con ceros)")

print(f"\nObservaciones sobre clase minoritaria 'negative':")
print(f"  - Es la clase más difícil de predecir debido al desbalanceo")
print(f"  - F1-Score: {report_dict['negative']['f1-score']:.4f}")
print(f"  - Se recomienda considerar técnicas de balanceo para mejorar rendimiento")

## 13. TAREA 4: Deep Learning - Preparación de Secuencias para LSTM/CNN

**Objetivo**: Preparar datos de texto en formato secuencial para modelos de Deep Learning (LSTM/CNN).

En esta sección vamos a:
1. Cargar datos preprocesados (tokens limpios sin lematizar)
2. Crear un Tokenizer de Keras para convertir texto a secuencias numéricas
3. Aplicar padding a las secuencias (max_length=100)
4. Crear matrices de embeddings pre-entrenadas usando Word2Vec y FastText
5. Aplicar splits Train/Val/Test de la TAREA 1
6. Verificar cobertura del vocabulario

**Nota importante**: Usamos tokens limpios (sin lematizar) porque los modelos Word2Vec y FastText fueron entrenados con palabras en su forma original.

In [ ]:
import pandas as pd
import numpy as np
import pickle
import ast
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec, FastText

# Establecer semilla
np.random.seed(42)

print("=" * 80)
print("TAREA 4: PREPARACIÓN DE SECUENCIAS PARA DEEP LEARNING")
print("=" * 80)

# PASO 1: CARGAR DATOS PREPROCESADOS
print("\n1. CARGANDO DATOS PREPROCESADOS")
print("-" * 80)

try:
    df_processed = pd.read_csv('data_processed/datos_preprocesados_completo.csv')
    print(f"✓ Datos cargados: {df_processed.shape}")
    print(f"Columnas disponibles: {list(df_processed.columns)}")
except FileNotFoundError:
    print("ERROR: No se encontró 'datos_preprocesados_completo.csv'")
    print("Por favor, ejecuta primero la sección 4 (Lemmatization y Stemming)")
    raise

# Verificar que tenemos la columna text_clean
if 'text_clean' not in df_processed.columns:
    print("ERROR: No se encontró columna 'text_clean'")
    print("Por favor, verifica que el preprocesamiento se completó correctamente")
    raise

# Verificar columna Etiqueta
if 'Etiqueta' not in df_processed.columns:
    print("NOTA: Creando columna 'Etiqueta' = 'correcta'")
    df_processed['Etiqueta'] = 'correcta'

print(f"\nPrimeras muestras de text_clean:")
print(df_processed['text_clean'].head(3))

# PASO 2: CREAR TOKENIZER Y CONVERTIR A SECUENCIAS
print("\n" + "=" * 80)
print("2. CREANDO TOKENIZER DE KERAS")
print("=" * 80)

# Parámetros
MAX_NUM_WORDS = 5000  # Vocabulario máximo
MAX_SEQUENCE_LENGTH = 100  # Longitud máxima de secuencias
OOV_TOKEN = '<OOV>'  # Token para palabras fuera de vocabulario

# Crear tokenizer
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(df_processed['text_clean'].values)

# Estadísticas del tokenizer
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # +1 por el índice 0 reservado

print(f"✓ Tokenizer creado")
print(f"  Vocabulario total: {len(word_index)} palabras")
print(f"  Vocabulario usado: {min(MAX_NUM_WORDS, vocab_size)} palabras")
print(f"  Token OOV: '{OOV_TOKEN}'")

# Mostrar algunas palabras del vocabulario
print(f"\nEjemplos de palabras en el vocabulario:")
sample_words = list(word_index.items())[:10]
for word, idx in sample_words:
    print(f"  {word}: {idx}")

# PASO 3: CONVERTIR TEXTOS A SECUENCIAS
print("\n" + "=" * 80)
print("3. CONVERTIR TEXTOS A SECUENCIAS NUMÉRICAS")
print("=" * 80)

sequences = tokenizer.texts_to_sequences(df_processed['text_clean'].values)

print(f"✓ Secuencias creadas: {len(sequences)} secuencias")

# Analizar longitudes de secuencias
sequence_lengths = [len(seq) for seq in sequences]
print(f"\nEstadísticas de longitud de secuencias:")
print(f"  Media: {np.mean(sequence_lengths):.2f}")
print(f"  Mediana: {np.median(sequence_lengths):.2f}")
print(f"  Mínima: {np.min(sequence_lengths)}")
print(f"  Máxima: {np.max(sequence_lengths)}")
print(f"  Percentil 95: {np.percentile(sequence_lengths, 95):.2f}")

# Ejemplo de conversión
print(f"\nEjemplo de conversión texto -> secuencia:")
example_text = df_processed['text_clean'].iloc[0]
example_seq = sequences[0]
print(f"  Texto: {example_text[:100]}...")
print(f"  Secuencia: {example_seq[:20]}...")

# PASO 4: APLICAR PADDING
print("\n" + "=" * 80)
print("4. APLICAR PADDING A LAS SECUENCIAS")
print("=" * 80)

padded_sequences = pad_sequences(
    sequences,
    maxlen=MAX_SEQUENCE_LENGTH,
    padding='post',
    truncating='post'
)

print(f"✓ Padding aplicado")
print(f"  Forma final: {padded_sequences.shape}")
print(f"  Longitud máxima: {MAX_SEQUENCE_LENGTH}")
print(f"  Tipo de padding: post (al final)")
print(f"  Tipo de truncado: post (desde el final)")

# Ejemplo de secuencia paddeada
print(f"\nEjemplo de secuencia paddeada:")
print(f"  Original (len={len(example_seq)}): {example_seq[:15]}...")
print(f"  Padded (len={len(padded_sequences[0])}): {padded_sequences[0][:15]}...")

# PASO 5: CARGAR MODELOS DE EMBEDDINGS PRE-ENTRENADOS
print("\n" + "=" * 80)
print("5. CARGAR MODELOS DE EMBEDDINGS PRE-ENTRENADOS")
print("=" * 80)

# Cargar Word2Vec
print("\n--- Word2Vec ---")
try:
    w2v_model = Word2Vec.load('models/word2vec_model.model')
    print(f"✓ Modelo Word2Vec cargado")
    print(f"  Dimensión: {w2v_model.wv.vector_size}")
    print(f"  Vocabulario: {len(w2v_model.wv)} palabras")
except FileNotFoundError:
    print("ERROR: No se encontró 'word2vec_model.model'")
    print("Por favor, ejecuta primero la sección 7 (Word Embeddings)")
    raise

# Cargar FastText
print("\n--- FastText ---")
try:
    ft_model = FastText.load('models/fasttext_model.model')
    print(f"✓ Modelo FastText cargado")
    print(f"  Dimensión: {ft_model.wv.vector_size}")
    print(f"  Vocabulario: {len(ft_model.wv)} palabras")
except FileNotFoundError:
    print("ERROR: No se encontró 'fasttext_model.model'")
    print("Por favor, ejecuta primero la sección 7 (Word Embeddings)")
    raise

# PASO 6: CREAR MATRICES DE EMBEDDINGS
print("\n" + "=" * 80)
print("6. CREAR MATRICES DE EMBEDDINGS PRE-ENTRENADAS")
print("=" * 80)

def create_embedding_matrix(word_index, embedding_model, embedding_dim, max_words):
    """
    Crea matriz de embeddings a partir de un modelo pre-entrenado
    
    Args:
        word_index: Diccionario palabra->índice del tokenizer
        embedding_model: Modelo Word2Vec o FastText
        embedding_dim: Dimensión de los embeddings
        max_words: Número máximo de palabras a incluir
    
    Returns:
        embedding_matrix: Matriz numpy de forma (vocab_size, embedding_dim)
    """
    vocab_size = min(len(word_index) + 1, max_words)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    found_words = 0
    missing_words = 0
    
    for word, idx in word_index.items():
        if idx >= max_words:
            continue
        
        try:
            # Intentar obtener el vector de la palabra
            embedding_vector = embedding_model.wv[word]
            embedding_matrix[idx] = embedding_vector
            found_words += 1
        except KeyError:
            # Palabra no encontrada, dejar como vector de ceros
            missing_words += 1
    
    return embedding_matrix, found_words, missing_words

# Crear matriz de Word2Vec
print("\n--- Matriz de embeddings Word2Vec ---")
embedding_dim_w2v = w2v_model.wv.vector_size
embedding_matrix_w2v, found_w2v, missing_w2v = create_embedding_matrix(
    word_index, w2v_model, embedding_dim_w2v, MAX_NUM_WORDS
)

print(f"✓ Matriz creada: {embedding_matrix_w2v.shape}")
print(f"  Palabras encontradas: {found_w2v}")
print(f"  Palabras no encontradas (OOV): {missing_w2v}")
print(f"  Cobertura: {found_w2v/(found_w2v+missing_w2v)*100:.2f}%")

# Crear matriz de FastText
print("\n--- Matriz de embeddings FastText ---")
embedding_dim_ft = ft_model.wv.vector_size
embedding_matrix_ft, found_ft, missing_ft = create_embedding_matrix(
    word_index, ft_model, embedding_dim_ft, MAX_NUM_WORDS
)

print(f"✓ Matriz creada: {embedding_matrix_ft.shape}")
print(f"  Palabras encontradas: {found_ft}")
print(f"  Palabras no encontradas (OOV): {missing_ft}")
print(f"  Cobertura: {found_ft/(found_ft+missing_ft)*100:.2f}%")

# PASO 7: PREPARAR SPLITS
print("\n" + "=" * 80)
print("7. PREPARAR SPLITS TRAIN/VAL/TEST")
print("=" * 80)

# Cargar dataset original para hacer match
df_original = pd.read_csv('data/initial_data.csv')

# Cargar splits
consistency_train = pd.read_csv('data_processed/consistency_train.csv')
consistency_val = pd.read_csv('data_processed/consistency_val.csv')
consistency_test = pd.read_csv('data_processed/consistency_test.csv')

sentiment_train = pd.read_csv('data_processed/sentiment_train.csv')
sentiment_val = pd.read_csv('data_processed/sentiment_val.csv')
sentiment_test = pd.read_csv('data_processed/sentiment_test.csv')

def get_split_indices(split_df, df_original_ref):
    """Obtiene índices del dataset original para un split"""
    indices = []
    split_sentences = split_df['Sentence'].values
    df_original_indexed = df_original_ref.reset_index(drop=True)
    
    for sentence in split_sentences:
        matches = df_original_indexed[df_original_indexed['Sentence'] == sentence].index
        if len(matches) > 0:
            indices.append(matches[0])
    
    return indices

# Obtener índices para cada split
print("\nObteniendo índices de splits...")

indices_consistency_train = get_split_indices(consistency_train, df_original)
indices_consistency_val = get_split_indices(consistency_val, df_original)
indices_consistency_test = get_split_indices(consistency_test, df_original)

indices_sentiment_train = get_split_indices(sentiment_train, df_original)
indices_sentiment_val = get_split_indices(sentiment_val, df_original)
indices_sentiment_test = get_split_indices(sentiment_test, df_original)

print(f"✓ Índices obtenidos:")
print(f"  Consistency - Train: {len(indices_consistency_train)}, Val: {len(indices_consistency_val)}, Test: {len(indices_consistency_test)}")
print(f"  Sentiment - Train: {len(indices_sentiment_train)}, Val: {len(indices_sentiment_val)}, Test: {len(indices_sentiment_test)}")

# PASO 8: GUARDAR TODOS LOS DATOS PREPARADOS
print("\n" + "=" * 80)
print("8. GUARDAR DATOS PREPARADOS")
print("=" * 80)

# Guardar secuencias paddeadas
np.savez_compressed(
    'data_processed/sequences_padded.npz',
    sequences=padded_sequences,
    # Guardar también los índices de splits
    consistency_train_idx=indices_consistency_train,
    consistency_val_idx=indices_consistency_val,
    consistency_test_idx=indices_consistency_test,
    sentiment_train_idx=indices_sentiment_train,
    sentiment_val_idx=indices_sentiment_val,
    sentiment_test_idx=indices_sentiment_test
)
print("✓ Secuencias paddeadas guardadas: data_processed/sequences_padded.npz")

# Guardar tokenizer
with open('models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("✓ Tokenizer guardado: models/tokenizer.pkl")

# Guardar matrices de embeddings
np.save('models/embedding_matrix_w2v.npy', embedding_matrix_w2v)
print("✓ Matriz Word2Vec guardada: models/embedding_matrix_w2v.npy")

np.save('models/embedding_matrix_ft.npy', embedding_matrix_ft)
print("✓ Matriz FastText guardada: models/embedding_matrix_ft.npy")

# Guardar información de configuración
config = {
    'max_num_words': MAX_NUM_WORDS,
    'max_sequence_length': MAX_SEQUENCE_LENGTH,
    'vocab_size': vocab_size,
    'embedding_dim_w2v': embedding_dim_w2v,
    'embedding_dim_ft': embedding_dim_ft,
    'w2v_coverage': found_w2v/(found_w2v+missing_w2v)*100,
    'ft_coverage': found_ft/(found_ft+missing_ft)*100
}

with open('models/sequences_config.pkl', 'wb') as f:
    pickle.dump(config, f)
print("✓ Configuración guardada: models/sequences_config.pkl")

# PASO 9: VERIFICACIÓN FINAL
print("\n" + "=" * 80)
print("9. VERIFICACIÓN Y RESUMEN")
print("=" * 80)

print(f"\n✓ Ejemplo de carga y uso:")
print(f"\n# Cargar secuencias")
print(f"data = np.load('data_processed/sequences_padded.npz')")
print(f"sequences = data['sequences']")
print(f"train_idx = data['consistency_train_idx']")
print(f"\n# Cargar tokenizer")
print(f"with open('models/tokenizer.pkl', 'rb') as f:")
print(f"    tokenizer = pickle.load(f)")
print(f"\n# Cargar matriz de embeddings")
print(f"embedding_matrix = np.load('models/embedding_matrix_w2v.npy')")

# Mostrar ejemplo práctico
print(f"\n✓ Verificación con datos de consistencia (train):")
X_train_cons = padded_sequences[indices_consistency_train]
y_train_cons = consistency_train['Etiqueta'].values
print(f"  X_train shape: {X_train_cons.shape}")
print(f"  y_train shape: {y_train_cons.shape}")
print(f"  Primera secuencia (primeros 20 tokens): {X_train_cons[0][:20]}")

# RESUMEN FINAL
print("\n" + "=" * 80)
print("RESUMEN FINAL - TAREA 4")
print("=" * 80)

print(f"\n✓ TAREA 4 COMPLETADA")
print(f"\nDatos preparados:")
print(f"  1. Secuencias numéricas paddeadas: {padded_sequences.shape}")
print(f"  2. Tokenizer de Keras (vocab={min(MAX_NUM_WORDS, vocab_size)})")
print(f"  3. Matriz Word2Vec: {embedding_matrix_w2v.shape} (cobertura: {found_w2v/(found_w2v+missing_w2v)*100:.2f}%)")
print(f"  4. Matriz FastText: {embedding_matrix_ft.shape} (cobertura: {found_ft/(found_ft+missing_ft)*100:.2f}%)")
print(f"  5. Índices de splits guardados para ambas tareas")

print(f"\nParámetros configurados:")
print(f"  - Vocabulario máximo: {MAX_NUM_WORDS}")
print(f"  - Longitud de secuencia: {MAX_SEQUENCE_LENGTH}")
print(f"  - Dimensión embeddings: {embedding_dim_w2v}")
print(f"  - Token OOV: {OOV_TOKEN}")

print(f"\nArchivos generados:")
print(f"  1. data_processed/sequences_padded.npz")
print(f"  2. models/tokenizer.pkl")
print(f"  3. models/embedding_matrix_w2v.npy")
print(f"  4. models/embedding_matrix_ft.npy")
print(f"  5. models/sequences_config.pkl")

print(f"\nCobertura de vocabulario:")
print(f"  - Word2Vec: {found_w2v} palabras encontradas, {missing_w2v} OOV ({found_w2v/(found_w2v+missing_w2v)*100:.2f}% cobertura)")
print(f"  - FastText: {found_ft} palabras encontradas, {missing_ft} OOV ({found_ft/(found_ft+missing_ft)*100:.2f}% cobertura)")
print(f"  - FastText tiene mejor cobertura por usar subword information")

print(f"\nLos datos están listos para entrenar modelos LSTM y CNN en las siguientes tareas.")

## 14. TAREA 5: Deep Learning - LSTM con Word2Vec (Consistencia)

**Objetivo**: Entrenar modelos LSTM para detección de consistencia comparando 3 configuraciones de embeddings.

En esta sección vamos a:
1. Cargar secuencias preparadas y matriz de embeddings Word2Vec
2. Crear arquitectura LSTM para clasificación binaria (correcta/incorrecta)
3. Entrenar con **3 configuraciones de embeddings**:
   - **Frozen**: Embeddings congelados (trainable=False) - usa conocimiento pre-entrenado
   - **Fine-tuned**: Embeddings entrenables (trainable=True) - adapta embeddings a la tarea
   - **From scratch**: Sin inicialización pre-entrenada - aprende desde cero
4. Comparar rendimiento, curvas de aprendizaje y tiempos de entrenamiento
5. Analizar cuándo usar cada configuración

**Tarea**: Detección de Consistencia (correcta vs incorrecta)  
**Arquitectura**: LSTM  
**Embeddings**: Word2Vec (100 dimensiones)

In [ ]:
import pandas as pd
import numpy as np
import pickle
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.keras.utils import to_categorical

# Establecer semillas para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

print("=" * 80)
print("TAREA 5: LSTM CON WORD2VEC - DETECCIÓN DE CONSISTENCIA")
print("=" * 80)

# PASO 1: CARGAR DATOS PREPARADOS
print("\n1. CARGANDO DATOS PREPARADOS")
print("-" * 80)

# Cargar secuencias paddeadas
data = np.load('data_processed/sequences_padded.npz')
sequences = data['sequences']
print(f"✓ Secuencias cargadas: {sequences.shape}")

# Cargar índices de splits para consistencia
train_idx = data['consistency_train_idx']
val_idx = data['consistency_val_idx']
test_idx = data['consistency_test_idx']

print(f"✓ Índices de splits cargados:")
print(f"  Train: {len(train_idx)} muestras")
print(f"  Validation: {len(val_idx)} muestras")
print(f"  Test: {len(test_idx)} muestras")

# Cargar matriz de embeddings Word2Vec
embedding_matrix_w2v = np.load('models/embedding_matrix_w2v.npy')
print(f"✓ Matriz de embeddings Word2Vec cargada: {embedding_matrix_w2v.shape}")

# Cargar configuración
with open('models/sequences_config.pkl', 'rb') as f:
    config = pickle.load(f)

MAX_SEQUENCE_LENGTH = config['max_sequence_length']
VOCAB_SIZE = config['vocab_size']
EMBEDDING_DIM = config['embedding_dim_w2v']

print(f"\nConfiguración:")
print(f"  Vocab size: {VOCAB_SIZE}")
print(f"  Sequence length: {MAX_SEQUENCE_LENGTH}")
print(f"  Embedding dim: {EMBEDDING_DIM}")

# PASO 2: PREPARAR DATOS DE ENTRENAMIENTO
print("\n2. PREPARAR DATOS PARA ENTRENAMIENTO")
print("-" * 80)

# Cargar etiquetas
consistency_train = pd.read_csv('data_processed/consistency_train.csv')
consistency_val = pd.read_csv('data_processed/consistency_val.csv')
consistency_test = pd.read_csv('data_processed/consistency_test.csv')

# Extraer X e y
X_train = sequences[train_idx]
X_val = sequences[val_idx]
X_test = sequences[test_idx]

# Convertir etiquetas a binario (correcta=0, incorrecta=1)
label_map = {'correcta': 0, 'incorrecta': 1}
y_train = np.array([label_map.get(label, 0) for label in consistency_train['Etiqueta'].values])
y_val = np.array([label_map.get(label, 0) for label in consistency_val['Etiqueta'].values])
y_test = np.array([label_map.get(label, 0) for label in consistency_test['Etiqueta'].values])

print(f"✓ Datos preparados:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"  X_test: {X_test.shape}, y_test: {y_test.shape}")

print(f"\nDistribución de clases en train:")
unique, counts = np.unique(y_train, return_counts=True)
for label, count in zip(unique, counts):
    label_name = 'correcta' if label == 0 else 'incorrecta'
    print(f"  {label_name}: {count} ({count/len(y_train)*100:.2f}%)")

# PASO 3: DEFINIR ARQUITECTURA LSTM
print("\n" + "=" * 80)
print("3. DEFINIR ARQUITECTURA LSTM")
print("=" * 80)

def create_lstm_model(vocab_size, embedding_dim, sequence_length, 
                      embedding_matrix=None, trainable=True):
    """
    Crea modelo LSTM para clasificación binaria
    
    Args:
        vocab_size: Tamaño del vocabulario
        embedding_dim: Dimensión de embeddings
        sequence_length: Longitud de secuencias
        embedding_matrix: Matriz de embeddings pre-entrenados (opcional)
        trainable: Si los embeddings son entrenables
    
    Returns:
        model: Modelo Keras compilado
    """
    model = Sequential([
        # Capa de Embedding
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            input_length=sequence_length,
            weights=[embedding_matrix] if embedding_matrix is not None else None,
            trainable=trainable,
            name='embedding'
        ),
        
        # Capa LSTM
        LSTM(64, return_sequences=False, name='lstm'),
        
        # Dropout para regularización
        Dropout(0.3, name='dropout'),
        
        # Capa densa intermedia
        Dense(32, activation='relu', name='dense'),
        
        # Capa de salida (clasificación binaria)
        Dense(1, activation='sigmoid', name='output')
    ])
    
    # Compilar modelo
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

print("✓ Arquitectura LSTM definida:")
print("\n  Embedding Layer → LSTM(64) → Dropout(0.3) → Dense(32) → Dense(1)")
print("\nParámetros:")
print("  - LSTM units: 64")
print("  - Dropout rate: 0.3")
print("  - Dense units: 32")
print("  - Activation: sigmoid (clasificación binaria)")
print("  - Optimizer: Adam (lr=0.001)")
print("  - Loss: binary_crossentropy")

# PASO 4: ENTRENAR MODELOS CON DIFERENTES CONFIGURACIONES
print("\n" + "=" * 80)
print("4. ENTRENAR MODELOS - 3 CONFIGURACIONES")
print("=" * 80)

# Configuración de entrenamiento
EPOCHS = 20
BATCH_SIZE = 32
EARLY_STOPPING_PATIENCE = 3

# Callback de early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=EARLY_STOPPING_PATIENCE,
    restore_best_weights=True,
    verbose=1
)

# Diccionario para almacenar resultados
results = {}
histories = {}

# CONFIGURACIÓN 1: FROZEN EMBEDDINGS
print("\n" + "-" * 80)
print("CONFIGURACIÓN 1: EMBEDDINGS CONGELADOS (FROZEN)")
print("-" * 80)
print("Los embeddings NO se actualizan durante el entrenamiento")
print("Ventaja: Más rápido, menos parámetros, usa conocimiento pre-entrenado")

model_frozen = create_lstm_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_matrix=embedding_matrix_w2v,
    trainable=False  # FROZEN
)

print(f"\nTotal de parámetros:")
model_frozen.summary()

print("\nEntrenando modelo FROZEN...")
start_time = time.time()
history_frozen = model_frozen.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)
training_time_frozen = time.time() - start_time

print(f"✓ Entrenamiento completado en {training_time_frozen:.2f}s")

# Evaluar en validation
y_val_pred_frozen = (model_frozen.predict(X_val) > 0.5).astype(int).flatten()
acc_frozen = accuracy_score(y_val, y_val_pred_frozen)
f1_frozen = f1_score(y_val, y_val_pred_frozen, average='binary')

results['frozen'] = {
    'model': model_frozen,
    'history': history_frozen,
    'accuracy': acc_frozen,
    'f1_score': f1_frozen,
    'training_time': training_time_frozen
}
histories['frozen'] = history_frozen

print(f"Resultados en Validation:")
print(f"  Accuracy: {acc_frozen:.4f}")
print(f"  F1-Score: {f1_frozen:.4f}")

# CONFIGURACIÓN 2: FINE-TUNED EMBEDDINGS
print("\n" + "-" * 80)
print("CONFIGURACIÓN 2: EMBEDDINGS FINE-TUNED")
print("-" * 80)
print("Los embeddings se actualizan durante el entrenamiento")
print("Ventaja: Adapta embeddings a la tarea específica, mejor rendimiento")

model_finetuned = create_lstm_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_matrix=embedding_matrix_w2v,
    trainable=True  # FINE-TUNED
)

print(f"\nTotal de parámetros:")
model_finetuned.summary()

print("\nEntrenando modelo FINE-TUNED...")
start_time = time.time()
history_finetuned = model_finetuned.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)
training_time_finetuned = time.time() - start_time

print(f"✓ Entrenamiento completado en {training_time_finetuned:.2f}s")

# Evaluar en validation
y_val_pred_finetuned = (model_finetuned.predict(X_val) > 0.5).astype(int).flatten()
acc_finetuned = accuracy_score(y_val, y_val_pred_finetuned)
f1_finetuned = f1_score(y_val, y_val_pred_finetuned, average='binary')

results['finetuned'] = {
    'model': model_finetuned,
    'history': history_finetuned,
    'accuracy': acc_finetuned,
    'f1_score': f1_finetuned,
    'training_time': training_time_finetuned
}
histories['finetuned'] = history_finetuned

print(f"Resultados en Validation:")
print(f"  Accuracy: {acc_finetuned:.4f}")
print(f"  F1-Score: {f1_finetuned:.4f}")

# CONFIGURACIÓN 3: FROM SCRATCH
print("\n" + "-" * 80)
print("CONFIGURACIÓN 3: FROM SCRATCH (SIN PRE-ENTRENAMIENTO)")
print("-" * 80)
print("Los embeddings se inicializan aleatoriamente y se aprenden desde cero")
print("Ventaja: No depende de embeddings externos, útil con vocabulario muy específico")

model_scratch = create_lstm_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_matrix=None,  # Sin pre-entrenamiento
    trainable=True
)

print(f"\nTotal de parámetros:")
model_scratch.summary()

print("\nEntrenando modelo FROM SCRATCH...")
start_time = time.time()
history_scratch = model_scratch.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping],
    verbose=1
)
training_time_scratch = time.time() - start_time

print(f"✓ Entrenamiento completado en {training_time_scratch:.2f}s")

# Evaluar en validation
y_val_pred_scratch = (model_scratch.predict(X_val) > 0.5).astype(int).flatten()
acc_scratch = accuracy_score(y_val, y_val_pred_scratch)
f1_scratch = f1_score(y_val, y_val_pred_scratch, average='binary')

results['scratch'] = {
    'model': model_scratch,
    'history': history_scratch,
    'accuracy': acc_scratch,
    'f1_score': f1_scratch,
    'training_time': training_time_scratch
}
histories['scratch'] = history_scratch

print(f"Resultados en Validation:")
print(f"  Accuracy: {acc_scratch:.4f}")
print(f"  F1-Score: {f1_scratch:.4f}")

# PASO 5: COMPARAR RESULTADOS
print("\n" + "=" * 80)
print("5. COMPARACIÓN DE RESULTADOS")
print("=" * 80)

comparison_df = pd.DataFrame({
    'Configuración': ['Frozen', 'Fine-tuned', 'From Scratch'],
    'Accuracy': [acc_frozen, acc_finetuned, acc_scratch],
    'F1-Score': [f1_frozen, f1_finetuned, f1_scratch],
    'Tiempo (s)': [training_time_frozen, training_time_finetuned, training_time_scratch]
})

print("\nTabla comparativa:")
print(comparison_df.to_string(index=False))

# Identificar mejor configuración
best_config = comparison_df.loc[comparison_df['F1-Score'].idxmax(), 'Configuración']
print(f"\n✓ Mejor configuración según F1-Score: {best_config}")

# PASO 6: VISUALIZAR CURVAS DE APRENDIZAJE
print("\n" + "=" * 80)
print("6. CURVAS DE APRENDIZAJE")
print("=" * 80)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Curvas de Aprendizaje - LSTM Word2Vec (Consistencia)', fontsize=16, fontweight='bold')

configs = ['frozen', 'finetuned', 'scratch']
titles = ['Frozen', 'Fine-tuned', 'From Scratch']

for idx, (config, title) in enumerate(zip(configs, titles)):
    history = histories[config]
    
    # Plot loss
    axes[idx].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[idx].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[idx].set_title(f'{title}\nF1: {results[config]["f1_score"]:.4f}')
    axes[idx].set_xlabel('Epoch')
    axes[idx].set_ylabel('Loss')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig('charts/07_lstm_w2v_learning_curves.png', dpi=300, bbox_inches='tight')
print("✓ Curvas guardadas: charts/07_lstm_w2v_learning_curves.png")
plt.show()

# PASO 7: EVALUACIÓN EN TEST SET
print("\n" + "=" * 80)
print("7. EVALUACIÓN EN TEST SET")
print("=" * 80)

best_config_key = best_config.lower().replace(' ', '')
best_model = results[best_config_key]['model']

y_test_pred = (best_model.predict(X_test) > 0.5).astype(int).flatten()

print(f"\nResultados del mejor modelo ({best_config}) en Test:")
print(classification_report(y_test, y_test_pred, target_names=['correcta', 'incorrecta']))

# Matriz de confusión
cm = confusion_matrix(y_test, y_test_pred)
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['correcta', 'incorrecta'],
            yticklabels=['correcta', 'incorrecta'])
ax.set_title(f'Matriz de Confusión - {best_config} (Test Set)')
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('charts/07_lstm_w2v_confusion_matrix.png', dpi=300, bbox_inches='tight')
print("✓ Matriz de confusión guardada: charts/07_lstm_w2v_confusion_matrix.png")
plt.show()

# PASO 8: GUARDAR MODELOS
print("\n" + "=" * 80)
print("8. GUARDAR MODELOS")
print("=" * 80)

model_frozen.save('models/w2v_lstm_frozen.h5')
print("✓ Modelo frozen guardado: models/w2v_lstm_frozen.h5")

model_finetuned.save('models/w2v_lstm_finetuned.h5')
print("✓ Modelo fine-tuned guardado: models/w2v_lstm_finetuned.h5")

model_scratch.save('models/w2v_lstm_scratch.h5')
print("✓ Modelo from scratch guardado: models/w2v_lstm_scratch.h5")

# Guardar resultados
comparison_df.to_csv('models/w2v_lstm_results.csv', index=False)
print("✓ Resultados guardados: models/w2v_lstm_results.csv")

# RESUMEN FINAL
print("\n" + "=" * 80)
print("RESUMEN FINAL - TAREA 5")
print("=" * 80)

print(f"\n✓ TAREA 5 COMPLETADA")
print(f"\nMejor configuración: {best_config}")
print(f"F1-Score (test): {f1_score(y_test, y_test_pred, average='binary'):.4f}")

print(f"\nComparación de configuraciones:")
print(comparison_df.to_string(index=False))

print(f"\nAnálisis y recomendaciones:")
print(f"\n1. FROZEN (Embeddings congelados):")
print(f"   • Más rápido de entrenar")
print(f"   • Menos parámetros (menor riesgo de overfitting)")
print(f"   • Útil cuando: dataset pequeño, recursos limitados")

print(f"\n2. FINE-TUNED (Embeddings ajustables):")
print(f"   • Mejor rendimiento en la tarea específica")
print(f"   • Adapta embeddings al dominio")
print(f"   • Útil cuando: dataset mediano/grande, task-specific vocabulary")

print(f"\n3. FROM SCRATCH (Sin pre-entrenamiento):")
print(f"   • Aprende todo desde cero")
print(f"   • Requiere más datos y tiempo")
print(f"   • Útil cuando: vocabulario muy específico, embeddings no relevantes")

print(f"\nArchivos generados:")
print(f"  1. 3 modelos entrenados (.h5)")
print(f"  2. Curvas de aprendizaje")
print(f"  3. Matriz de confusión")
print(f"  4. Tabla de resultados (CSV)")